In [1]:
import pandas as pd
import glob
from matplotlib import pyplot as plt
import numpy as np
import re
#import bamboolib

In [2]:
import os
current_directory = os.getcwd()
#print("Current directory : ", current_directory)

aymeric =  "/home/aymeric/python-scripts/espadon/data/" #aymeric
jp = '~/Dropbox/Mac/Desktop/CRD Anses/all3/' # Jean Philippe
jp_index = '~/Dropbox/Mac/Desktop/CRD Anses/code/indexation_results/' # Jean Philippe index

if 'aymeric' in current_directory:
    path_base = aymeric

elif 'Mac' in current_directory:
    path_base = jp
elif 'd:/Projects' in current_directory:
    path_base = "d:/Projects/Medialab/"

#print("Path base : ", path_base)

In [3]:
# Using '*' pattern 
#print('\nNamed with wildcard *:')
i=-1
dic_id={}
for x in [x for x in pd.read_csv(glob.glob(f'{path_base}sm/*.csv')[0]).columns if 'id' in x]:
    dic_id[x]=str
dic_id
for name in glob.glob(f'{path_base}/sm/*.csv'):
    #print (name)
    i+=1
    df0=pd.read_csv(name,dtype=dic_id)
    if i>0:
        df=pd.concat([df,df0])
    else:
        df=df0

/home/aymeric/anaconda3/envs/analyse_tweets/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (39) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


KeyboardInterrupt: 

In [13]:
df0=df.drop_duplicates()

In [14]:
df0["text"] =  df0.text.apply(lambda x : x.replace('\r', ' '))
df0["text"] =  df0.text.apply(lambda x : x.replace('\n', ' '))

# Détection des biomarqueurs dans les tweets



## Méthode initiale

Afin de compter les occurrences des biomarqueurs dans les tweets, la méthode initiale consistait à rechercher simplement les chaînes de caractères correspondant aux noms des biomarqueurs en prenant ou non en compte la casse. Pour rappel, les biomarqueurs pris en compte sont: *ROS1, ALK, EXON, EGFR, KRAS, NTRK, BRAF, MET, RET, HER2*.

Toutefois, cette première méthode conduisait à prendre en compte de nombreux faux-amis dans le cas de certains biomarqueurs. Par exemple, suivant cette méthode, "RETweet", "WALK", "VincenTRK" ou "METASTASIS" étaient compté comme une occurrence de "RET", "ALK", "NTRK" ou "MET" respectivement.




In [15]:
biom=['ROS1', 'ALK','EXON', 'EGFR','KRAS','NTRK','BRAF',"MET",'RET',"HER2"]#
dfb = df0.copy()

for x in biom:
    if x in ['MET','RET','ALK']:
        dfb[x]=dfb.text.str.contains(x, case=True)
    else:
        dfb[x]=dfb.text.str.contains(x, case=False)
    
    

## Définition d'expressions régulières plus complexes

Des expressions régulières plus "complexes" ont alors été définies pour exclure les faux-amis évoqués ci-dessus et d'autres du décomptes des occurrences. Le script utilisé et affiché dans la cellule ci-dessous fonctionne de la façon suivante:

1. Pour chaque tweet, on commence par remplacer plusieurs signes de ponctuations ("/", ".", ",", "-") par des espaces.


2. Une fois le remplacement effectué, chaque tweets est découpé afin d'obtenir la liste de toutes les chaînes de caractère précédées et suivies par un espace (dans le script ci-dessous, cette liste est nommée "tt").


3. puis on recherche la présence de chaque biormaqueur au sein de cette liste selon des expressions régulières. Par exemple, dans le cas du biomarqueur "MET", nous avons utilisé l'expression régulière suivante (avec x est égale à "MET" et le signe "|" qui signifie "ou"):

`"{x}[(+)]" | "[#@+]{x}[^AaSsUuEeYyHh]" | "[#@+]{x}$" | "^{x}[^AaSsUuEeYyHh]" | "^{x}$"`
    
   
C'est-à-dire que pour chaque terme de la liste "tt", celui-ci sera considéré comme une occurrence du biomarqueur "MET" si, et seulement si "MET" :
    
- est suivi du signe +" 
    
- **ou** est précédé par un hashtag ou un arobase et suivi par un ou plusieurs caractères sauf les lettres majuscules ou minuscules *a, s, u, e, y, h*. 
    
- **ou** est précédé par un hashtag, un arobase ou le signe "+" et n'est suivi par aucun caractères
    
- **ou** n'est précédé par aucun caractère et n'est pas suivi par les lettres *a, s, u, e, y, h* majuscules ou minuscules". Dans ce cas, l'expression "METUProg" n'est pas prise en compte à la différence de "METmuts" 
    
- **ou** n'est précédé ni suivi de caractères.
    
Si certaines règles sont communes à l'ensemble des biomarqueurs, d'autres sont spécifiques. Par exemple, dans le cas du biomarqueur "RET", les lettres  *a, s, u, e, y, h* sont remplacées par *a, e, i, o, u, r, h, z, w* pour éviter que des mots comme "retweet" soient comptés. À noter également que dans le cas des biomarqueurs "ALK", "MET" et "RET", on ne prend en compte que les formes en majuscule. Pour les autres, l'algorithme de recherche est insensible à la casse.

Enfin, ces expressions régulières sont recherchées à l'aide de la fonction "search" du module *re* du langage Python. Cette fonction retourne "vrai" si l'expression est trouvée, "faux" dans le cas contraire. Par ailleurs, elle s'arrête à la première expression trouvée. Autrement dit, la fonction indique simplement si l'expression est présente quelque soit le nombre de fois où elle apparaît dans le tweet. Le tableau ci-après illustre le résultat obtenu pour une dizaine de phrases avec chacune des deux méthodes (sans ou avec les expressions régulières).

La principale limite de cette seconde méthode est qu'elle conduit à définir de nombreuses exceptions pour éviter que des "vrais positifs" soit traités comme des "faux positifs". Ainsi, la règle qui consiste à ne pas compter les chaînes de caractère constituées de MET suivi d'un "a", comme *METastasis*, a pour effet également d'exclure les formes du types *METamp*. La liste des formes exclues est placée à la fin de ce document.


:::{admonition} Illustration de l'algorythme

1. Le tweet initial : 

>"MET copy number as a secondary driver of EGFR TKI resistance in EGFR-mutant NSCLC http://bit.ly/2HfGHjn #editorial #lcsm"</p>
    
2. Le tweet nettoyé (les "/", ".", "," et "-" sont remplacés par des espaces) : 

> "MET copy number as a secondary driver of EGFR TKI resistance in EGFR mutant NSCLC http:  bit ly 2HfGHjn #editorial #lcsm"
    
3. On crée une liste de toutes les chaînes de caractère précédées et suivies par une espace :

> ['MET', 'copy', 'number', 'as', 'a', 'secondary', 'driver', 'of', 'EGFR', 'TKI', 'resistance', 'in', 'EGFR', 'mutant', 'NSCLC', 'http:', '', 'bit', 'ly', '2HfGHjn', '#editorial', '#lcsm']
    
4. La recherche du biomarqueur se fait ensuite à partir de la liste des termes entre crochets ci-dessus. Dans cet exemple, on a bien le terme "MET". On constate que le tweet fait égalment référence au biomarqueur EGFR.

:::



In [16]:
c = 0
c2 = 0
x = 'MET'
dict_sentence = []
index_name = []
for n, t in enumerate(df0["text"][0:100000]):
    liste = []
    if len(dict_sentence) < 16 :
        #liste.append(t)
        tt = t.replace("/", " ").replace("-", " ").replace("(", " ").replace(")", " ").replace(".", " ").replace(",", " ").split(" ")
        ttt = t.replace("/", " ").replace("-", " ").replace("(", " ").replace(")", " ").replace(".", " ").replace(",", " ")
        
        marker = [f"{x}[(+)]", f"^[#@+]{x}[^AaSsUuEeYyHh]", f"[#@+]{x}$", f"^{x}[^AaSsUuEeYyHh]", f"^{x}$"]
        marker2 = f"{x}"

        word = [w for w in tt if re.search('|'.join(marker), w)]
        word2 = [w for w in tt if re.search(marker2, w)]
        if len(word2) > 0:
            index_name.append(t)
            liste.append(True)
            list_word = "; ".join(word2)
            liste.append(list_word)
            if len(word) > 0:
                c = c + 1
                liste.append(True)
                list_word = "; ".join(word)
                liste.append(list_word)
                
            else:
                liste.append(False)
                liste.append(np.nan)
            dict_sentence.append(liste)
        else:
            if len(word) > 0:
                c = c + 1
                index_name.append(t)
                liste.append(False)
                liste.append(np.nan)
                liste.append(True)
                list_word = "; ".join(word)
                liste.append(list_word)
                dict_sentence.append(liste)
                
            else :
                pass
            

    else:
        pass
    
    




In [17]:
cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'color: black; text-align:left'
}

                   
df_exemple = pd.DataFrame(dict_sentence, 
                          index=pd.Index(index_name, name='Tweets :'),
                          columns=pd.MultiIndex.from_product([['Sans régex', 'Avec régex'],
                                    ['Contient MET', 'Forme détectée']], names = ["Méthodes :", ""]))
s = df_exemple.style.format(precision=0, na_rep='Aucune', 
                formatter={('Sans régex', 'Avec régex'): lambda x: "$ {:,.1f}".format(x*-1e6)
                          })

s.set_table_styles([cell_hover, index_names, headers])
s.set_caption("Comparaison de la méthode initiale et de celle utilisant les régex sur le biomarqueur MET")\
 .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: top'
 }], overwrite=False)

In [19]:

biom=['ROS1', 'ALK','EXON', 'EGFR','KRAS','NTRK','BRAF',"MET",'RET',"HER2"]#
dict_marker = {'ROS1':[], 'ALK':[],'EXON':[], 'EGFR':[],'KRAS':[],'NTRK':[],'BRAF':[],"MET":[],'RET':[],"HER2":[]}
dict_marker2 = {'ROS1':[], 'ALK':[],'EXON':[], 'EGFR':[],'KRAS':[],'NTRK':[],'BRAF':[],"MET":[],'RET':[],"HER2":[]}
list_marker = []

list_word2 = []
for n, t in enumerate(df0['text']):
    
    list_word = []
    for ponct in [",", ".", ";", "-", "/", "(", ")", "[", "]", "{", "}", "?", "!", ":","\"", "\'", "\|", "_",
                 "&", "%"]:
        t = t.replace(ponct, " ")
    tt = t.split(" ")
    id_text = df0["id"].iloc[n]
    list_word.append(n)
    list_word.append(id_text)
    for x in biom:
        list_word2 = []
        if x == "MET":
            marker = [f"{x}[(+)]", f"^[#@+]{x}[^AaSsUuEeYyHh]", f"[#@+]{x}$", f"^{x}[^AaSsUuEeYyHh]",  f"^{x}$"]
            word = [w for w in tt if re.search('|'.join(marker), w)]
            word2 = [w for w in tt if re.search(f'{x}', w) and w not in word]
            
            
        elif x == 'RET':
            marker = [f"{x}[(+)]", f"[#@+]{x}[^wWaeiouAEIOUrRhHzZ]", f"[#@+]{x}$", f"^{x}[^wWaeiouAEIOUrRhHzZ]", f"^{x}$"]
            word = [w for w in tt if re.search('|'.join(marker), w)]
            word2 = [w for w in tt if re.search(f'{x}', w) and w not in word]
        
        elif x == 'ALK':
            marker = [f"{x}[(+)]", f"[#@+]{x}",  f"[^TtWw]{x}$", f"[^TtWw]{x}", f"^{x}",  f"^{x}$"]
            word = [w for w in tt if re.search('|'.join(marker), w)]
            word2 = [w for w in tt if re.search(f'{x}', w) and w not in word]
            
        elif x == 'NTRK':
            marker = [f"{x}[(+)]", f"[#@]{x}",  f"^{x}", f"[^(vince)]{x}", f"^{x}$"]
            word = [w for w in tt if re.search('|'.join(marker), w, re.IGNORECASE)]
            word2 = [w for w in tt if re.search(f'{x}', w,  re.IGNORECASE) and w not in word]

                
        else:
            marker = [f"{x}[(-)]", f"{x}[(+)]", f"[#@]{x}", f"^{x}", f"{x}"]
            word = [w for w in tt if re.search('|'.join(marker), w, re.IGNORECASE)]
            word2 = [w for w in tt if re.search(f'{x}', w,  re.IGNORECASE) and w not in word]
        
        if len(word) > 0:
            list_word.append(1)
            for y in word:
                list_word2.append(y)
        else:
            list_word.append(0)
        
        
        
        previous_list_word = dict_marker[x]
        current_list_word = list_word2
        upgraded_list_word = previous_list_word + current_list_word
        dict_marker[x] = [x for x in set(upgraded_list_word)]
        

        if len(word2) > 0:
            #print(n)
            #print(t)
            #print(word)
            #print(word2)
            previous_list_word2 = dict_marker2[x]
            upgraded_list_word2 = previous_list_word2 + word2
            dict_marker2[x] = [x for x in set(upgraded_list_word2)]
        else:
            pass
        
    
      
    list_marker.append(list_word)

        
            
    

In [20]:
columns_name = ['index', 'id', 'ROS1', 'ALK','EXON', 'EGFR','KRAS','NTRK','BRAF',"MET",'RET',"HER2"]
df_marker = pd.DataFrame(np.array(list_marker), columns = columns_name)

In [21]:
type_to_change={'id': str, 'ROS1':int,'ALK': int,'EXON':int, 'EGFR':int,'KRAS':int,'NTRK':int,
                'BRAF':int,'MET':int,'RET':int,'HER2':int}
df_marker = df_marker.drop(columns = ["index"]).astype(type_to_change)

In [22]:

comparaison = []
for x in biom:
    liste_count = []
    df_m = df_marker.loc[df_marker[x]==1]
    dfb_b = dfb.loc[dfb[x]==True]
    liste_count.append(x)
    liste_count.append(len(df_m))
    liste_count.append(len(dfb_b))
    liste_count.append(len(df_m) - len(dfb_b))
    comparaison.append(liste_count)
    #comparaison[x] = liste_count


Le tableau ci-dessous donne le nombre d'occurrences retrouvées pour chacun des marqueurs avec les deux méthodes.

In [59]:
columns_name = ["Biomarqueur", "Avec expressions régulières", "Sans expression régulière", "Ecart"]
count_marker = pd.DataFrame(np.array(comparaison), columns = columns_name).sort_values("Ecart", ascending = True)
#count_marker.to_csv("comparaison_des_regles_de_matching.csv", sep = ',')
count_marker_style = count_marker.style.format(precision=0, na_rep='')\
.set_caption("Le nombre de marqueurs trouvés selon la méthode utilisée")\
 .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: top ; color: black ; font-size : 14pt'
 }], overwrite=False)
count_marker_style

,Biomarqueur,Avec expressions régulières,Sans expression régulière,Ecart
1,ALK,7164,7325,-161
5,NTRK,638,4541,-3903
7,MET,1326,5458,-4132
8,RET,1218,1690,-472
0,ROS1,2874,2874,0
2,EXON,1478,1478,0
3,EGFR,11154,11154,0
4,KRAS,2568,2568,0
6,BRAF,2775,2775,0
9,HER2,11691,11691,0


<style type="text/css">#T_7d20b_ caption {  caption-side: top;  color: black;  font-size: 14pt;}</style><table id="T_7d20b_">  <caption>Le nombre de marqueurs trouvés selon la méthode utilisée</caption>  <thead>    <tr>      <th class="blank level0" >&nbsp;</th>      <th class="col_heading level0 col0" >Biomarqueur</th>      <th class="col_heading level0 col1" >Avec expressions régulières</th>      <th class="col_heading level0 col2" >Sans expression régulière</th>      <th class="col_heading level0 col3" >Ecart</th>    </tr>  </thead>  <tbody>    <tr>      <th id="T_7d20b_level0_row0" class="row_heading level0 row0" >1</th>      <td id="T_7d20b_row0_col0" class="data row0 col0" >ALK</td>      <td id="T_7d20b_row0_col1" class="data row0 col1" >7164</td>      <td id="T_7d20b_row0_col2" class="data row0 col2" >7325</td>      <td id="T_7d20b_row0_col3" class="data row0 col3" >-161</td>    </tr>    <tr>      <th id="T_7d20b_level0_row1" class="row_heading level0 row1" >5</th>      <td id="T_7d20b_row1_col0" class="data row1 col0" >NTRK</td>      <td id="T_7d20b_row1_col1" class="data row1 col1" >638</td>      <td id="T_7d20b_row1_col2" class="data row1 col2" >4541</td>      <td id="T_7d20b_row1_col3" class="data row1 col3" >-3903</td>    </tr>    <tr>      <th id="T_7d20b_level0_row2" class="row_heading level0 row2" >7</th>      <td id="T_7d20b_row2_col0" class="data row2 col0" >MET</td>      <td id="T_7d20b_row2_col1" class="data row2 col1" >1326</td>      <td id="T_7d20b_row2_col2" class="data row2 col2" >5458</td>      <td id="T_7d20b_row2_col3" class="data row2 col3" >-4132</td>    </tr>    <tr>      <th id="T_7d20b_level0_row3" class="row_heading level0 row3" >8</th>      <td id="T_7d20b_row3_col0" class="data row3 col0" >RET</td>      <td id="T_7d20b_row3_col1" class="data row3 col1" >1218</td>      <td id="T_7d20b_row3_col2" class="data row3 col2" >1690</td>      <td id="T_7d20b_row3_col3" class="data row3 col3" >-472</td>    </tr>    <tr>      <th id="T_7d20b_level0_row4" class="row_heading level0 row4" >0</th>      <td id="T_7d20b_row4_col0" class="data row4 col0" >ROS1</td>      <td id="T_7d20b_row4_col1" class="data row4 col1" >2874</td>      <td id="T_7d20b_row4_col2" class="data row4 col2" >2874</td>      <td id="T_7d20b_row4_col3" class="data row4 col3" >0</td>    </tr>    <tr>      <th id="T_7d20b_level0_row5" class="row_heading level0 row5" >2</th>      <td id="T_7d20b_row5_col0" class="data row5 col0" >EXON</td>      <td id="T_7d20b_row5_col1" class="data row5 col1" >1478</td>      <td id="T_7d20b_row5_col2" class="data row5 col2" >1478</td>      <td id="T_7d20b_row5_col3" class="data row5 col3" >0</td>    </tr>    <tr>      <th id="T_7d20b_level0_row6" class="row_heading level0 row6" >3</th>      <td id="T_7d20b_row6_col0" class="data row6 col0" >EGFR</td>      <td id="T_7d20b_row6_col1" class="data row6 col1" >11154</td>      <td id="T_7d20b_row6_col2" class="data row6 col2" >11154</td>      <td id="T_7d20b_row6_col3" class="data row6 col3" >0</td>    </tr>    <tr>      <th id="T_7d20b_level0_row7" class="row_heading level0 row7" >4</th>      <td id="T_7d20b_row7_col0" class="data row7 col0" >KRAS</td>      <td id="T_7d20b_row7_col1" class="data row7 col1" >2568</td>      <td id="T_7d20b_row7_col2" class="data row7 col2" >2568</td>      <td id="T_7d20b_row7_col3" class="data row7 col3" >0</td>    </tr>    <tr>      <th id="T_7d20b_level0_row8" class="row_heading level0 row8" >6</th>      <td id="T_7d20b_row8_col0" class="data row8 col0" >BRAF</td>      <td id="T_7d20b_row8_col1" class="data row8 col1" >2775</td>      <td id="T_7d20b_row8_col2" class="data row8 col2" >2775</td>      <td id="T_7d20b_row8_col3" class="data row8 col3" >0</td>    </tr>    <tr>      <th id="T_7d20b_level0_row9" class="row_heading level0 row9" >9</th>      <td id="T_7d20b_row9_col0" class="data row9 col0" >HER2</td>      <td id="T_7d20b_row9_col1" class="data row9 col1" >11691</td>      <td id="T_7d20b_row9_col2" class="data row9 col2" >11691</td>      <td id="T_7d20b_row9_col3" class="data row9 col3" >0</td>    </tr>  </tbody></table>

In [25]:
df_marker["nb_of_biomarker"] = df_marker[biom].sum(axis=1)

In [26]:
df01 = df0[["id"]]
df_marker1 =df_marker[["id"]]

In [27]:
df00 =  df0.merge(df_marker, how = "inner", on = ["id"])

In [28]:
d = {}
for m in biom:
    l = [x for x in dict_marker2[m]]
    d[m] = l

In [40]:
i = -1
for m in biom:
    
    i+=1
    dfbiom01= pd.DataFrame({m : d[m]}).drop_duplicates(subset=[m])
    dfbiom01 = dfbiom01.reset_index()

    if i>0:
        dfbiom1 = dfbiom1.merge(dfbiom01, on = ["index"], how = "outer")
    else:
        dfbiom1 = dfbiom01

In [42]:
dfbiom1 = dfbiom1.drop(columns = ["index","ROS1","EXON","EGFR","KRAS", "BRAF", "HER2"])


In [60]:
dfbiom1_style = dfbiom1.style.format(precision=0, na_rep='')\
.set_table_styles([cell_hover, index_names, headers])\
.set_properties(**{'text-align': 'left'})\
.set_caption("Les formes comptées comme occurrences par la méthode 1 et exclues avec la méthodes 2")\
 .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: top ; color: black ; font-size : 14pt'
 }], overwrite=False)

dfbiom1_style

,ALK,NTRK,MET,RET
0,@NEWSTALK1010,@IMFjimMYELOM@VincentRK,U7METv,RETURN
1,WALK,@vincentRK,⁦@METUPUKorg,RETRO
2,TALKED,⁦⁦@VincentRK⁩,#SABRCOMET,RETEVMO
3,WALKED,@VincentRK,@METHILRENUKA,RETROSPECTIVE
4,#spacewALK,vincentrk,#COMETS,RETWEETERS
5,NEWSTALK,⁦@VincentRK⁩,#METexon14skipping,#RETRenegades
6,#LUNGFORCEWALKBOSTON,⁦⁦@VincentRK⁩⁦@MatthewHoMD⁩,#METup,BRET
7,TALKIBC,📊@VincentRK,#METUPUK,SECRET
8,#BTWALK,@myelomaMD @VincentRK BCJ,TAXOMET,RETAIN
9,#tALKpositiveNSCLC,@vicentRK,#METASTATIC,#RETrenegades


'<style type="text/css">#T_0def3_ td:hover {  background-color: #ffffb3;}#T_0def3_ .index_name {  font-style: italic;  color: darkgrey;  font-weight: normal;}#T_0def3_ th:not(.index_name) {  color: black;  text-align: left;}#T_0def3_ caption {  caption-side: top;  color: black;  font-size: 14pt;}#T_0def3_row0_col0, #T_0def3_row0_col1, #T_0def3_row0_col2, #T_0def3_row0_col3, #T_0def3_row1_col0, #T_0def3_row1_col1, #T_0def3_row1_col2, #T_0def3_row1_col3, #T_0def3_row2_col0, #T_0def3_row2_col1, #T_0def3_row2_col2, #T_0def3_row2_col3, #T_0def3_row3_col0, #T_0def3_row3_col1, #T_0def3_row3_col2, #T_0def3_row3_col3, #T_0def3_row4_col0, #T_0def3_row4_col1, #T_0def3_row4_col2, #T_0def3_row4_col3, #T_0def3_row5_col0, #T_0def3_row5_col1, #T_0def3_row5_col2, #T_0def3_row5_col3, #T_0def3_row6_col0, #T_0def3_row6_col1, #T_0def3_row6_col2, #T_0def3_row6_col3, #T_0def3_row7_col0, #T_0def3_row7_col1, #T_0def3_row7_col2, #T_0def3_row7_col3, #T_0def3_row8_col0, #T_0def3_row8_col1, #T_0def3_row8_col2, #T_0def3_row8_col3, #T_0def3_row9_col0, #T_0def3_row9_col1, #T_0def3_row9_col2, #T_0def3_row9_col3, #T_0def3_row10_col0, #T_0def3_row10_col1, #T_0def3_row10_col2, #T_0def3_row10_col3, #T_0def3_row11_col0, #T_0def3_row11_col1, #T_0def3_row11_col2, #T_0def3_row11_col3, #T_0def3_row12_col0, #T_0def3_row12_col1, #T_0def3_row12_col2, #T_0def3_row12_col3, #T_0def3_row13_col0, #T_0def3_row13_col1, #T_0def3_row13_col2, #T_0def3_row13_col3, #T_0def3_row14_col0, #T_0def3_row14_col1, #T_0def3_row14_col2, #T_0def3_row14_col3, #T_0def3_row15_col0, #T_0def3_row15_col1, #T_0def3_row15_col2, #T_0def3_row15_col3, #T_0def3_row16_col0, #T_0def3_row16_col1, #T_0def3_row16_col2, #T_0def3_row16_col3, #T_0def3_row17_col0, #T_0def3_row17_col1, #T_0def3_row17_col2, #T_0def3_row17_col3, #T_0def3_row18_col0, #T_0def3_row18_col1, #T_0def3_row18_col2, #T_0def3_row18_col3, #T_0def3_row19_col0, #T_0def3_row19_col1, #T_0def3_row19_col2, #T_0def3_row19_col3, #T_0def3_row20_col0, #T_0def3_row20_col1, #T_0def3_row20_col2, #T_0def3_row20_col3, #T_0def3_row21_col0, #T_0def3_row21_col1, #T_0def3_row21_col2, #T_0def3_row21_col3, #T_0def3_row22_col0, #T_0def3_row22_col1, #T_0def3_row22_col2, #T_0def3_row22_col3, #T_0def3_row23_col0, #T_0def3_row23_col1, #T_0def3_row23_col2, #T_0def3_row23_col3, #T_0def3_row24_col0, #T_0def3_row24_col1, #T_0def3_row24_col2, #T_0def3_row24_col3, #T_0def3_row25_col0, #T_0def3_row25_col1, #T_0def3_row25_col2, #T_0def3_row25_col3, #T_0def3_row26_col0, #T_0def3_row26_col1, #T_0def3_row26_col2, #T_0def3_row26_col3, #T_0def3_row27_col0, #T_0def3_row27_col1, #T_0def3_row27_col2, #T_0def3_row27_col3, #T_0def3_row28_col0, #T_0def3_row28_col1, #T_0def3_row28_col2, #T_0def3_row28_col3, #T_0def3_row29_col0, #T_0def3_row29_col1, #T_0def3_row29_col2, #T_0def3_row29_col3, #T_0def3_row30_col0, #T_0def3_row30_col1, #T_0def3_row30_col2, #T_0def3_row30_col3, #T_0def3_row31_col0, #T_0def3_row31_col1, #T_0def3_row31_col2, #T_0def3_row31_col3, #T_0def3_row32_col0, #T_0def3_row32_col1, #T_0def3_row32_col2, #T_0def3_row32_col3, #T_0def3_row33_col0, #T_0def3_row33_col1, #T_0def3_row33_col2, #T_0def3_row33_col3, #T_0def3_row34_col0, #T_0def3_row34_col1, #T_0def3_row34_col2, #T_0def3_row34_col3, #T_0def3_row35_col0, #T_0def3_row35_col1, #T_0def3_row35_col2, #T_0def3_row35_col3, #T_0def3_row36_col0, #T_0def3_row36_col1, #T_0def3_row36_col2, #T_0def3_row36_col3, #T_0def3_row37_col0, #T_0def3_row37_col1, #T_0def3_row37_col2, #T_0def3_row37_col3, #T_0def3_row38_col0, #T_0def3_row38_col1, #T_0def3_row38_col2, #T_0def3_row38_col3, #T_0def3_row39_col0, #T_0def3_row39_col1, #T_0def3_row39_col2, #T_0def3_row39_col3, #T_0def3_row40_col0, #T_0def3_row40_col1, #T_0def3_row40_col2, #T_0def3_row40_col3, #T_0def3_row41_col0, #T_0def3_row41_col1, #T_0def3_row41_col2, #T_0def3_row41_col3, #T_0def3_row42_col0, #T_0def3_row42_col1, #T_0def3_row42_col2, #T_0def3_row42_col3, #T_0def3_row43_col0, #T_0def3_row43_col1, #T_0def3_row43_col2, #T_0def3_row43_col3, #T_0def3_row44_col0, #T_0def3_row44_col1, #T_0def3_row44_col2, #T_0def3_row44_col3, #T_0def3_row45_col0, #T_0def3_row45_col1, #T_0def3_row45_col2, #T_0def3_row45_col3, #T_0def3_row46_col0, #T_0def3_row46_col1, #T_0def3_row46_col2, #T_0def3_row46_col3, #T_0def3_row47_col0, #T_0def3_row47_col1, #T_0def3_row47_col2, #T_0def3_row47_col3, #T_0def3_row48_col0, #T_0def3_row48_col1, #T_0def3_row48_col2, #T_0def3_row48_col3, #T_0def3_row49_col0, #T_0def3_row49_col1, #T_0def3_row49_col2, #T_0def3_row49_col3, #T_0def3_row50_col0, #T_0def3_row50_col1, #T_0def3_row50_col2, #T_0def3_row50_col3, #T_0def3_row51_col0, #T_0def3_row51_col1, #T_0def3_row51_col2, #T_0def3_row51_col3, #T_0def3_row52_col0, #T_0def3_row52_col1, #T_0def3_row52_col2, #T_0def3_row52_col3, #T_0def3_row53_col0, #T_0def3_row53_col1, #T_0def3_row53_col2, #T_0def3_row53_col3, #T_0def3_row54_col0, #T_0def3_row54_col1, #T_0def3_row54_col2, #T_0def3_row54_col3, #T_0def3_row55_col0, #T_0def3_row55_col1, #T_0def3_row55_col2, #T_0def3_row55_col3, #T_0def3_row56_col0, #T_0def3_row56_col1, #T_0def3_row56_col2, #T_0def3_row56_col3, #T_0def3_row57_col0, #T_0def3_row57_col1, #T_0def3_row57_col2, #T_0def3_row57_col3, #T_0def3_row58_col0, #T_0def3_row58_col1, #T_0def3_row58_col2, #T_0def3_row58_col3, #T_0def3_row59_col0, #T_0def3_row59_col1, #T_0def3_row59_col2, #T_0def3_row59_col3, #T_0def3_row60_col0, #T_0def3_row60_col1, #T_0def3_row60_col2, #T_0def3_row60_col3, #T_0def3_row61_col0, #T_0def3_row61_col1, #T_0def3_row61_col2, #T_0def3_row61_col3, #T_0def3_row62_col0, #T_0def3_row62_col1, #T_0def3_row62_col2, #T_0def3_row62_col3, #T_0def3_row63_col0, #T_0def3_row63_col1, #T_0def3_row63_col2, #T_0def3_row63_col3, #T_0def3_row64_col0, #T_0def3_row64_col1, #T_0def3_row64_col2, #T_0def3_row64_col3, #T_0def3_row65_col0, #T_0def3_row65_col1, #T_0def3_row65_col2, #T_0def3_row65_col3, #T_0def3_row66_col0, #T_0def3_row66_col1, #T_0def3_row66_col2, #T_0def3_row66_col3, #T_0def3_row67_col0, #T_0def3_row67_col1, #T_0def3_row67_col2, #T_0def3_row67_col3, #T_0def3_row68_col0, #T_0def3_row68_col1, #T_0def3_row68_col2, #T_0def3_row68_col3, #T_0def3_row69_col0, #T_0def3_row69_col1, #T_0def3_row69_col2, #T_0def3_row69_col3, #T_0def3_row70_col0, #T_0def3_row70_col1, #T_0def3_row70_col2, #T_0def3_row70_col3, #T_0def3_row71_col0, #T_0def3_row71_col1, #T_0def3_row71_col2, #T_0def3_row71_col3, #T_0def3_row72_col0, #T_0def3_row72_col1, #T_0def3_row72_col2, #T_0def3_row72_col3, #T_0def3_row73_col0, #T_0def3_row73_col1, #T_0def3_row73_col2, #T_0def3_row73_col3, #T_0def3_row74_col0, #T_0def3_row74_col1, #T_0def3_row74_col2, #T_0def3_row74_col3, #T_0def3_row75_col0, #T_0def3_row75_col1, #T_0def3_row75_col2, #T_0def3_row75_col3, #T_0def3_row76_col0, #T_0def3_row76_col1, #T_0def3_row76_col2, #T_0def3_row76_col3, #T_0def3_row77_col0, #T_0def3_row77_col1, #T_0def3_row77_col2, #T_0def3_row77_col3, #T_0def3_row78_col0, #T_0def3_row78_col1, #T_0def3_row78_col2, #T_0def3_row78_col3, #T_0def3_row79_col0, #T_0def3_row79_col1, #T_0def3_row79_col2, #T_0def3_row79_col3, #T_0def3_row80_col0, #T_0def3_row80_col1, #T_0def3_row80_col2, #T_0def3_row80_col3, #T_0def3_row81_col0, #T_0def3_row81_col1, #T_0def3_row81_col2, #T_0def3_row81_col3, #T_0def3_row82_col0, #T_0def3_row82_col1, #T_0def3_row82_col2, #T_0def3_row82_col3, #T_0def3_row83_col0, #T_0def3_row83_col1, #T_0def3_row83_col2, #T_0def3_row83_col3, #T_0def3_row84_col0, #T_0def3_row84_col1, #T_0def3_row84_col2, #T_0def3_row84_col3, #T_0def3_row85_col0, #T_0def3_row85_col1, #T_0def3_row85_col2, #T_0def3_row85_col3, #T_0def3_row86_col0, #T_0def3_row86_col1, #T_0def3_row86_col2, #T_0def3_row86_col3, #T_0def3_row87_col0, #T_0def3_row87_col1, #T_0def3_row87_col2, #T_0def3_row87_col3, #T_0def3_row88_col0, #T_0def3_row88_col1, #T_0def3_row88_col2, #T_0def3_row88_col3, #T_0def3_row89_col0, #T_0def3_row89_col1, #T_0def3_row89_col2, #T_0def3_row89_col3, #T_0def3_row90_col0, #T_0def3_row90_col1, #T_0def3_row90_col2, #T_0def3_row90_col3, #T_0def3_row91_col0, #T_0def3_row91_col1, #T_0def3_row91_col2, #T_0def3_row91_col3, #T_0def3_row92_col0, #T_0def3_row92_col1, #T_0def3_row92_col2, #T_0def3_row92_col3, #T_0def3_row93_col0, #T_0def3_row93_col1, #T_0def3_row93_col2, #T_0def3_row93_col3, #T_0def3_row94_col0, #T_0def3_row94_col1, #T_0def3_row94_col2, #T_0def3_row94_col3, #T_0def3_row95_col0, #T_0def3_row95_col1, #T_0def3_row95_col2, #T_0def3_row95_col3, #T_0def3_row96_col0, #T_0def3_row96_col1, #T_0def3_row96_col2, #T_0def3_row96_col3, #T_0def3_row97_col0, #T_0def3_row97_col1, #T_0def3_row97_col2, #T_0def3_row97_col3, #T_0def3_row98_col0, #T_0def3_row98_col1, #T_0def3_row98_col2, #T_0def3_row98_col3, #T_0def3_row99_col0, #T_0def3_row99_col1, #T_0def3_row99_col2, #T_0def3_row99_col3, #T_0def3_row100_col0, #T_0def3_row100_col1, #T_0def3_row100_col2, #T_0def3_row100_col3, #T_0def3_row101_col0, #T_0def3_row101_col1, #T_0def3_row101_col2, #T_0def3_row101_col3, #T_0def3_row102_col0, #T_0def3_row102_col1, #T_0def3_row102_col2, #T_0def3_row102_col3, #T_0def3_row103_col0, #T_0def3_row103_col1, #T_0def3_row103_col2, #T_0def3_row103_col3, #T_0def3_row104_col0, #T_0def3_row104_col1, #T_0def3_row104_col2, #T_0def3_row104_col3, #T_0def3_row105_col0, #T_0def3_row105_col1, #T_0def3_row105_col2, #T_0def3_row105_col3, #T_0def3_row106_col0, #T_0def3_row106_col1, #T_0def3_row106_col2, #T_0def3_row106_col3, #T_0def3_row107_col0, #T_0def3_row107_col1, #T_0def3_row107_col2, #T_0def3_row107_col3, #T_0def3_row108_col0, #T_0def3_row108_col1, #T_0def3_row108_col2, #T_0def3_row108_col3, #T_0def3_row109_col0, #T_0def3_row109_col1, #T_0def3_row109_col2, #T_0def3_row109_col3, #T_0def3_row110_col0, #T_0def3_row110_col1, #T_0def3_row110_col2, #T_0def3_row110_col3, #T_0def3_row111_col0, #T_0def3_row111_col1, #T_0def3_row111_col2, #T_0def3_row111_col3, #T_0def3_row112_col0, #T_0def3_row112_col1, #T_0def3_row112_col2, #T_0def3_row112_col3, #T_0def3_row113_col0, #T_0def3_row113_col1, #T_0def3_row113_col2, #T_0def3_row113_col3, #T_0def3_row114_col0, #T_0def3_row114_col1, #T_0def3_row114_col2, #T_0def3_row114_col3, #T_0def3_row115_col0, #T_0def3_row115_col1, #T_0def3_row115_col2, #T_0def3_row115_col3, #T_0def3_row116_col0, #T_0def3_row116_col1, #T_0def3_row116_col2, #T_0def3_row116_col3, #T_0def3_row117_col0, #T_0def3_row117_col1, #T_0def3_row117_col2, #T_0def3_row117_col3, #T_0def3_row118_col0, #T_0def3_row118_col1, #T_0def3_row118_col2, #T_0def3_row118_col3, #T_0def3_row119_col0, #T_0def3_row119_col1, #T_0def3_row119_col2, #T_0def3_row119_col3, #T_0def3_row120_col0, #T_0def3_row120_col1, #T_0def3_row120_col2, #T_0def3_row120_col3, #T_0def3_row121_col0, #T_0def3_row121_col1, #T_0def3_row121_col2, #T_0def3_row121_col3, #T_0def3_row122_col0, #T_0def3_row122_col1, #T_0def3_row122_col2, #T_0def3_row122_col3, #T_0def3_row123_col0, #T_0def3_row123_col1, #T_0def3_row123_col2, #T_0def3_row123_col3, #T_0def3_row124_col0, #T_0def3_row124_col1, #T_0def3_row124_col2, #T_0def3_row124_col3, #T_0def3_row125_col0, #T_0def3_row125_col1, #T_0def3_row125_col2, #T_0def3_row125_col3, #T_0def3_row126_col0, #T_0def3_row126_col1, #T_0def3_row126_col2, #T_0def3_row126_col3, #T_0def3_row127_col0, #T_0def3_row127_col1, #T_0def3_row127_col2, #T_0def3_row127_col3, #T_0def3_row128_col0, #T_0def3_row128_col1, #T_0def3_row128_col2, #T_0def3_row128_col3, #T_0def3_row129_col0, #T_0def3_row129_col1, #T_0def3_row129_col2, #T_0def3_row129_col3, #T_0def3_row130_col0, #T_0def3_row130_col1, #T_0def3_row130_col2, #T_0def3_row130_col3, #T_0def3_row131_col0, #T_0def3_row131_col1, #T_0def3_row131_col2, #T_0def3_row131_col3, #T_0def3_row132_col0, #T_0def3_row132_col1, #T_0def3_row132_col2, #T_0def3_row132_col3, #T_0def3_row133_col0, #T_0def3_row133_col1, #T_0def3_row133_col2, #T_0def3_row133_col3, #T_0def3_row134_col0, #T_0def3_row134_col1, #T_0def3_row134_col2, #T_0def3_row134_col3, #T_0def3_row135_col0, #T_0def3_row135_col1, #T_0def3_row135_col2, #T_0def3_row135_col3, #T_0def3_row136_col0, #T_0def3_row136_col1, #T_0def3_row136_col2, #T_0def3_row136_col3, #T_0def3_row137_col0, #T_0def3_row137_col1, #T_0def3_row137_col2, #T_0def3_row137_col3, #T_0def3_row138_col0, #T_0def3_row138_col1, #T_0def3_row138_col2, #T_0def3_row138_col3, #T_0def3_row139_col0, #T_0def3_row139_col1, #T_0def3_row139_col2, #T_0def3_row139_col3, #T_0def3_row140_col0, #T_0def3_row140_col1, #T_0def3_row140_col2, #T_0def3_row140_col3, #T_0def3_row141_col0, #T_0def3_row141_col1, #T_0def3_row141_col2, #T_0def3_row141_col3, #T_0def3_row142_col0, #T_0def3_row142_col1, #T_0def3_row142_col2, #T_0def3_row142_col3, #T_0def3_row143_col0, #T_0def3_row143_col1, #T_0def3_row143_col2, #T_0def3_row143_col3, #T_0def3_row144_col0, #T_0def3_row144_col1, #T_0def3_row144_col2, #T_0def3_row144_col3, #T_0def3_row145_col0, #T_0def3_row145_col1, #T_0def3_row145_col2, #T_0def3_row145_col3, #T_0def3_row146_col0, #T_0def3_row146_col1, #T_0def3_row146_col2, #T_0def3_row146_col3, #T_0def3_row147_col0, #T_0def3_row147_col1, #T_0def3_row147_col2, #T_0def3_row147_col3, #T_0def3_row148_col0, #T_0def3_row148_col1, #T_0def3_row148_col2, #T_0def3_row148_col3, #T_0def3_row149_col0, #T_0def3_row149_col1, #T_0def3_row149_col2, #T_0def3_row149_col3, #T_0def3_row150_col0, #T_0def3_row150_col1, #T_0def3_row150_col2, #T_0def3_row150_col3, #T_0def3_row151_col0, #T_0def3_row151_col1, #T_0def3_row151_col2, #T_0def3_row151_col3, #T_0def3_row152_col0, #T_0def3_row152_col1, #T_0def3_row152_col2, #T_0def3_row152_col3, #T_0def3_row153_col0, #T_0def3_row153_col1, #T_0def3_row153_col2, #T_0def3_row153_col3, #T_0def3_row154_col0, #T_0def3_row154_col1, #T_0def3_row154_col2, #T_0def3_row154_col3, #T_0def3_row155_col0, #T_0def3_row155_col1, #T_0def3_row155_col2, #T_0def3_row155_col3, #T_0def3_row156_col0, #T_0def3_row156_col1, #T_0def3_row156_col2, #T_0def3_row156_col3, #T_0def3_row157_col0, #T_0def3_row157_col1, #T_0def3_row157_col2, #T_0def3_row157_col3, #T_0def3_row158_col0, #T_0def3_row158_col1, #T_0def3_row158_col2, #T_0def3_row158_col3, #T_0def3_row159_col0, #T_0def3_row159_col1, #T_0def3_row159_col2, #T_0def3_row159_col3, #T_0def3_row160_col0, #T_0def3_row160_col1, #T_0def3_row160_col2, #T_0def3_row160_col3, #T_0def3_row161_col0, #T_0def3_row161_col1, #T_0def3_row161_col2, #T_0def3_row161_col3, #T_0def3_row162_col0, #T_0def3_row162_col1, #T_0def3_row162_col2, #T_0def3_row162_col3, #T_0def3_row163_col0, #T_0def3_row163_col1, #T_0def3_row163_col2, #T_0def3_row163_col3, #T_0def3_row164_col0, #T_0def3_row164_col1, #T_0def3_row164_col2, #T_0def3_row164_col3, #T_0def3_row165_col0, #T_0def3_row165_col1, #T_0def3_row165_col2, #T_0def3_row165_col3, #T_0def3_row166_col0, #T_0def3_row166_col1, #T_0def3_row166_col2, #T_0def3_row166_col3, #T_0def3_row167_col0, #T_0def3_row167_col1, #T_0def3_row167_col2, #T_0def3_row167_col3, #T_0def3_row168_col0, #T_0def3_row168_col1, #T_0def3_row168_col2, #T_0def3_row168_col3, #T_0def3_row169_col0, #T_0def3_row169_col1, #T_0def3_row169_col2, #T_0def3_row169_col3, #T_0def3_row170_col0, #T_0def3_row170_col1, #T_0def3_row170_col2, #T_0def3_row170_col3, #T_0def3_row171_col0, #T_0def3_row171_col1, #T_0def3_row171_col2, #T_0def3_row171_col3, #T_0def3_row172_col0, #T_0def3_row172_col1, #T_0def3_row172_col2, #T_0def3_row172_col3, #T_0def3_row173_col0, #T_0def3_row173_col1, #T_0def3_row173_col2, #T_0def3_row173_col3, #T_0def3_row174_col0, #T_0def3_row174_col1, #T_0def3_row174_col2, #T_0def3_row174_col3, #T_0def3_row175_col0, #T_0def3_row175_col1, #T_0def3_row175_col2, #T_0def3_row175_col3, #T_0def3_row176_col0, #T_0def3_row176_col1, #T_0def3_row176_col2, #T_0def3_row176_col3, #T_0def3_row177_col0, #T_0def3_row177_col1, #T_0def3_row177_col2, #T_0def3_row177_col3, #T_0def3_row178_col0, #T_0def3_row178_col1, #T_0def3_row178_col2, #T_0def3_row178_col3, #T_0def3_row179_col0, #T_0def3_row179_col1, #T_0def3_row179_col2, #T_0def3_row179_col3, #T_0def3_row180_col0, #T_0def3_row180_col1, #T_0def3_row180_col2, #T_0def3_row180_col3, #T_0def3_row181_col0, #T_0def3_row181_col1, #T_0def3_row181_col2, #T_0def3_row181_col3, #T_0def3_row182_col0, #T_0def3_row182_col1, #T_0def3_row182_col2, #T_0def3_row182_col3, #T_0def3_row183_col0, #T_0def3_row183_col1, #T_0def3_row183_col2, #T_0def3_row183_col3, #T_0def3_row184_col0, #T_0def3_row184_col1, #T_0def3_row184_col2, #T_0def3_row184_col3, #T_0def3_row185_col0, #T_0def3_row185_col1, #T_0def3_row185_col2, #T_0def3_row185_col3, #T_0def3_row186_col0, #T_0def3_row186_col1, #T_0def3_row186_col2, #T_0def3_row186_col3, #T_0def3_row187_col0, #T_0def3_row187_col1, #T_0def3_row187_col2, #T_0def3_row187_col3, #T_0def3_row188_col0, #T_0def3_row188_col1, #T_0def3_row188_col2, #T_0def3_row188_col3, #T_0def3_row189_col0, #T_0def3_row189_col1, #T_0def3_row189_col2, #T_0def3_row189_col3, #T_0def3_row190_col0, #T_0def3_row190_col1, #T_0def3_row190_col2, #T_0def3_row190_col3, #T_0def3_row191_col0, #T_0def3_row191_col1, #T_0def3_row191_col2, #T_0def3_row191_col3, #T_0def3_row192_col0, #T_0def3_row192_col1, #T_0def3_row192_col2, #T_0def3_row192_col3, #T_0def3_row193_col0, #T_0def3_row193_col1, #T_0def3_row193_col2, #T_0def3_row193_col3, #T_0def3_row194_col0, #T_0def3_row194_col1, #T_0def3_row194_col2, #T_0def3_row194_col3, #T_0def3_row195_col0, #T_0def3_row195_col1, #T_0def3_row195_col2, #T_0def3_row195_col3, #T_0def3_row196_col0, #T_0def3_row196_col1, #T_0def3_row196_col2, #T_0def3_row196_col3, #T_0def3_row197_col0, #T_0def3_row197_col1, #T_0def3_row197_col2, #T_0def3_row197_col3, #T_0def3_row198_col0, #T_0def3_row198_col1, #T_0def3_row198_col2, #T_0def3_row198_col3, #T_0def3_row199_col0, #T_0def3_row199_col1, #T_0def3_row199_col2, #T_0def3_row199_col3, #T_0def3_row200_col0, #T_0def3_row200_col1, #T_0def3_row200_col2, #T_0def3_row200_col3, #T_0def3_row201_col0, #T_0def3_row201_col1, #T_0def3_row201_col2, #T_0def3_row201_col3, #T_0def3_row202_col0, #T_0def3_row202_col1, #T_0def3_row202_col2, #T_0def3_row202_col3, #T_0def3_row203_col0, #T_0def3_row203_col1, #T_0def3_row203_col2, #T_0def3_row203_col3, #T_0def3_row204_col0, #T_0def3_row204_col1, #T_0def3_row204_col2, #T_0def3_row204_col3, #T_0def3_row205_col0, #T_0def3_row205_col1, #T_0def3_row205_col2, #T_0def3_row205_col3, #T_0def3_row206_col0, #T_0def3_row206_col1, #T_0def3_row206_col2, #T_0def3_row206_col3, #T_0def3_row207_col0, #T_0def3_row207_col1, #T_0def3_row207_col2, #T_0def3_row207_col3 {  text-align: left;}</style><table id="T_0def3_">  <caption>Les formes comptées comme occurrences par la méthode 1 et exclues avec la méthodes 2</caption>  <thead>    <tr>      <th class="blank level0" >&nbsp;</th>      <th class="col_heading level0 col0" >ALK</th>      <th class="col_heading level0 col1" >NTRK</th>      <th class="col_heading level0 col2" >MET</th>      <th class="col_heading level0 col3" >RET</th>    </tr>  </thead>  <tbody>    <tr>      <th id="T_0def3_level0_row0" class="row_heading level0 row0" >0</th>      <td id="T_0def3_row0_col0" class="data row0 col0" >@NEWSTALK1010</td>      <td id="T_0def3_row0_col1" class="data row0 col1" >@IMFjimMYELOM@VincentRK</td>      <td id="T_0def3_row0_col2" class="data row0 col2" >U7METv</td>      <td id="T_0def3_row0_col3" class="data row0 col3" >RETURN</td>    </tr>    <tr>      <th id="T_0def3_level0_row1" class="row_heading level0 row1" >1</th>      <td id="T_0def3_row1_col0" class="data row1 col0" >WALK</td>      <td id="T_0def3_row1_col1" class="data row1 col1" >@vincentRK</td>      <td id="T_0def3_row1_col2" class="data row1 col2" >\u2066@METUPUKorg</td>      <td id="T_0def3_row1_col3" class="data row1 col3" >RETRO</td>    </tr>    <tr>      <th id="T_0def3_level0_row2" class="row_heading level0 row2" >2</th>      <td id="T_0def3_row2_col0" class="data row2 col0" >TALKED</td>      <td id="T_0def3_row2_col1" class="data row2 col1" >\u2066\u2066@VincentRK\u2069</td>      <td id="T_0def3_row2_col2" class="data row2 col2" >#SABRCOMET</td>      <td id="T_0def3_row2_col3" class="data row2 col3" >RETEVMO</td>    </tr>    <tr>      <th id="T_0def3_level0_row3" class="row_heading level0 row3" >3</th>      <td id="T_0def3_row3_col0" class="data row3 col0" >WALKED</td>      <td id="T_0def3_row3_col1" class="data row3 col1" >@VincentRK</td>      <td id="T_0def3_row3_col2" class="data row3 col2" >@METHILRENUKA</td>      <td id="T_0def3_row3_col3" class="data row3 col3" >RETROSPECTIVE</td>    </tr>    <tr>      <th id="T_0def3_level0_row4" class="row_heading level0 row4" >4</th>      <td id="T_0def3_row4_col0" class="data row4 col0" >#spacewALK</td>      <td id="T_0def3_row4_col1" class="data row4 col1" >vincentrk</td>      <td id="T_0def3_row4_col2" class="data row4 col2" >#COMETS</td>      <td id="T_0def3_row4_col3" class="data row4 col3" >RETWEETERS</td>    </tr>    <tr>      <th id="T_0def3_level0_row5" class="row_heading level0 row5" >5</th>      <td id="T_0def3_row5_col0" class="data row5 col0" >NEWSTALK</td>      <td id="T_0def3_row5_col1" class="data row5 col1" >\u2066@VincentRK\u2069</td>      <td id="T_0def3_row5_col2" class="data row5 col2" >#METexon14skipping</td>      <td id="T_0def3_row5_col3" class="data row5 col3" >#RETRenegades</td>    </tr>    <tr>      <th id="T_0def3_level0_row6" class="row_heading level0 row6" >6</th>      <td id="T_0def3_row6_col0" class="data row6 col0" >#LUNGFORCEWALKBOSTON</td>      <td id="T_0def3_row6_col1" class="data row6 col1" >\u2066\u2066@VincentRK\u2069\u2066@MatthewHoMD\u2069</td>      <td id="T_0def3_row6_col2" class="data row6 col2" >#METup</td>      <td id="T_0def3_row6_col3" class="data row6 col3" >BRET</td>    </tr>    <tr>      <th id="T_0def3_level0_row7" class="row_heading level0 row7" >7</th>      <td id="T_0def3_row7_col0" class="data row7 col0" >TALKIBC</td>      <td id="T_0def3_row7_col1" class="data row7 col1" >📊@VincentRK</td>      <td id="T_0def3_row7_col2" class="data row7 col2" >#METUPUK</td>      <td id="T_0def3_row7_col3" class="data row7 col3" >SECRET</td>    </tr>    <tr>      <th id="T_0def3_level0_row8" class="row_heading level0 row8" >8</th>      <td id="T_0def3_row8_col0" class="data row8 col0" >#BTWALK</td>      <td id="T_0def3_row8_col1" class="data row8 col1" >\xa0@myelomaMD\xa0\xa0@VincentRK\xa0\xa0BCJ</td>      <td id="T_0def3_row8_col2" class="data row8 col2" >TAXOMET</td>      <td id="T_0def3_row8_col3" class="data row8 col3" >RETAIN</td>    </tr>    <tr>      <th id="T_0def3_level0_row9" class="row_heading level0 row9" >9</th>      <td id="T_0def3_row9_col0" class="data row9 col0" >#tALKpositiveNSCLC</td>      <td id="T_0def3_row9_col1" class="data row9 col1" >@vicentRK</td>      <td id="T_0def3_row9_col2" class="data row9 col2" >#METASTATIC</td>      <td id="T_0def3_row9_col3" class="data row9 col3" >#RETrenegades</td>    </tr>    <tr>      <th id="T_0def3_level0_row10" class="row_heading level0 row10" >10</th>      <td id="T_0def3_row10_col0" class="data row10 col0" >TALK</td>      <td id="T_0def3_row10_col1" class="data row10 col1" >\xa0@DrOlaLandgren\xa0@VincentRK\xa0et</td>      <td id="T_0def3_row10_col2" class="data row10 col2" >#METEOR</td>      <td id="T_0def3_row10_col3" class="data row10 col3" >FRET</td>    </tr>    <tr>      <th id="T_0def3_level0_row11" class="row_heading level0 row11" >11</th>      <td id="T_0def3_row11_col0" class="data row11 col0" >HFTCWALK</td>      <td id="T_0def3_row11_col1" class="data row11 col1" >@Rfonsi1@VincentRK</td>      <td id="T_0def3_row11_col2" class="data row11 col2" >METABEAM</td>      <td id="T_0def3_row11_col3" class="data row11 col3" >#RETWEET</td>    </tr>    <tr>      <th id="T_0def3_level0_row12" class="row_heading level0 row12" >12</th>      <td id="T_0def3_row12_col0" class="data row12 col0" >NEWSTALK1010</td>      <td id="T_0def3_row12_col1" class="data row12 col1" >@vincentrk</td>      <td id="T_0def3_row12_col2" class="data row12 col2" >CHEMOMETER</td>      <td id="T_0def3_row12_col3" class="data row12 col3" >CARETAKER</td>    </tr>    <tr>      <th id="T_0def3_level0_row13" class="row_heading level0 row13" >13</th>      <td id="T_0def3_row13_col0" class="data row13 col0" >#AABBPEPTALK</td>      <td id="T_0def3_row13_col1" class="data row13 col1" >@VincentRKOncology</td>      <td id="T_0def3_row13_col2" class="data row13 col2" >@METUPorg’s</td>      <td id="T_0def3_row13_col3" class="data row13 col3" >CURETakedaMMRF</td>    </tr>    <tr>      <th id="T_0def3_level0_row14" class="row_heading level0 row14" >14</th>      <td id="T_0def3_row14_col0" class="data row14 col0" >SIDEWALK</td>      <td id="T_0def3_row14_col1" class="data row14 col1" >@VincentRK’s</td>      <td id="T_0def3_row14_col2" class="data row14 col2" >IMETELSTAT</td>      <td id="T_0def3_row14_col3" class="data row14 col3" >GAVRETO</td>    </tr>    <tr>      <th id="T_0def3_level0_row15" class="row_heading level0 row15" >15</th>      <td id="T_0def3_row15_col0" class="data row15 col0" >CURETALK</td>      <td id="T_0def3_row15_col1" class="data row15 col1" >@SusanNTRKers</td>      <td id="T_0def3_row15_col2" class="data row15 col2" >#METAvivorgrants</td>      <td id="T_0def3_row15_col3" class="data row15 col3" >DIRETAMENTE</td>    </tr>    <tr>      <th id="T_0def3_level0_row16" class="row_heading level0 row16" >16</th>      <td id="T_0def3_row16_col0" class="data row16 col0" >@TEALWALK</td>      <td id="T_0def3_row16_col1" class="data row16 col1" >\u2066@VincentRK</td>      <td id="T_0def3_row16_col2" class="data row16 col2" >METHODIST</td>      <td id="T_0def3_row16_col3" class="data row16 col3" >RETRÔ</td>    </tr>    <tr>      <th id="T_0def3_level0_row17" class="row_heading level0 row17" >17</th>      <td id="T_0def3_row17_col0" class="data row17 col0" >#RealTALK</td>      <td id="T_0def3_row17_col1" class="data row17 col1" >“@VincentRK</td>      <td id="T_0def3_row17_col2" class="data row17 col2" >ANZAMET</td>      <td id="T_0def3_row17_col3" class="data row17 col3" >#RETRenegrades</td>    </tr>    <tr>      <th id="T_0def3_level0_row18" class="row_heading level0 row18" >18</th>      <td id="T_0def3_row18_col0" class="data row18 col0" >TALKING</td>      <td id="T_0def3_row18_col1" class="data row18 col1" >@VincentRK|Lisa</td>      <td id="T_0def3_row18_col2" class="data row18 col2" >METHODS</td>      <td id="T_0def3_row18_col3" class="data row18 col3" >the\u202fRET\u202fgene</td>    </tr>    <tr>      <th id="T_0def3_level0_row19" class="row_heading level0 row19" >19</th>      <td id="T_0def3_row19_col0" class="data row19 col0" >2021wALK</td>      <td id="T_0def3_row19_col1" class="data row19 col1" >VincentRK</td>      <td id="T_0def3_row19_col2" class="data row19 col2" >#COMET2017Indy</td>      <td id="T_0def3_row19_col3" class="data row19 col3" >2RET9vQ</td>    </tr>    <tr>      <th id="T_0def3_level0_row20" class="row_heading level0 row20" >20</th>      <td id="T_0def3_row20_col0" class="data row20 col0" >#LUNGFORCEWALK</td>      <td id="T_0def3_row20_col1" class="data row20 col1" >\xa0@VincentRK\xa0</td>      <td id="T_0def3_row20_col2" class="data row20 col2" >👇🏻@METUPorg</td>      <td id="T_0def3_row20_col3" class="data row20 col3" >RETRACTED</td>    </tr>    <tr>      <th id="T_0def3_level0_row21" class="row_heading level0 row21" >21</th>      <td id="T_0def3_row21_col0" class="data row21 col0" >#TEDTALK</td>      <td id="T_0def3_row21_col1" class="data row21 col1" ></td>      <td id="T_0def3_row21_col2" class="data row21 col2" >PRIMETIME</td>      <td id="T_0def3_row21_col3" class="data row21 col3" >RETHINK</td>    </tr>    <tr>      <th id="T_0def3_level0_row22" class="row_heading level0 row22" >22</th>      <td id="T_0def3_row22_col0" class="data row22 col0" >#STALKLUNG</td>      <td id="T_0def3_row22_col1" class="data row22 col1" ></td>      <td id="T_0def3_row22_col2" class="data row22 col2" >#POKEMET</td>      <td id="T_0def3_row22_col3" class="data row22 col3" >@RETRenegades</td>    </tr>    <tr>      <th id="T_0def3_level0_row23" class="row_heading level0 row23" >23</th>      <td id="T_0def3_row23_col0" class="data row23 col0" >#PDXBTWALK</td>      <td id="T_0def3_row23_col1" class="data row23 col1" ></td>      <td id="T_0def3_row23_col2" class="data row23 col2" >METUPPositionpaperonClinicalTrials</td>      <td id="T_0def3_row23_col3" class="data row23 col3" >ARETA</td>    </tr>    <tr>      <th id="T_0def3_level0_row24" class="row_heading level0 row24" >24</th>      <td id="T_0def3_row24_col0" class="data row24 col0" >#ECOTALKS</td>      <td id="T_0def3_row24_col1" class="data row24 col1" ></td>      <td id="T_0def3_row24_col2" class="data row24 col2" >#cMET</td>      <td id="T_0def3_row24_col3" class="data row24 col3" >#RETHINKCANCER</td>    </tr>    <tr>      <th id="T_0def3_level0_row25" class="row_heading level0 row25" >25</th>      <td id="T_0def3_row25_col0" class="data row25 col0" >#BRAINTUMORWALK</td>      <td id="T_0def3_row25_col1" class="data row25 col1" ></td>      <td id="T_0def3_row25_col2" class="data row25 col2" >BOMET</td>      <td id="T_0def3_row25_col3" class="data row25 col3" >#RETWEEET</td>    </tr>    <tr>      <th id="T_0def3_level0_row26" class="row_heading level0 row26" >26</th>      <td id="T_0def3_row26_col0" class="data row26 col0" >@BoardWALK4Cure</td>      <td id="T_0def3_row26_col1" class="data row26 col1" ></td>      <td id="T_0def3_row26_col2" class="data row26 col2" >\u2066@METUPUKorg\u2069</td>      <td id="T_0def3_row26_col3" class="data row26 col3" >CONCRETE</td>    </tr>    <tr>      <th id="T_0def3_level0_row27" class="row_heading level0 row27" >27</th>      <td id="T_0def3_row27_col0" class="data row27 col0" >WALKING</td>      <td id="T_0def3_row27_col1" class="data row27 col1" ></td>      <td id="T_0def3_row27_col2" class="data row27 col2" >#METex14</td>      <td id="T_0def3_row27_col3" class="data row27 col3" >CURETALK</td>    </tr>    <tr>      <th id="T_0def3_level0_row28" class="row_heading level0 row28" >28</th>      <td id="T_0def3_row28_col0" class="data row28 col0" >#SPACEWALK</td>      <td id="T_0def3_row28_col1" class="data row28 col1" ></td>      <td id="T_0def3_row28_col2" class="data row28 col2" >PAPMET</td>      <td id="T_0def3_row28_col3" class="data row28 col3" >⚡#RETWEET</td>    </tr>    <tr>      <th id="T_0def3_level0_row29" class="row_heading level0 row29" >29</th>      <td id="T_0def3_row29_col0" class="data row29 col0" >@MHFTALKBACK</td>      <td id="T_0def3_row29_col1" class="data row29 col1" ></td>      <td id="T_0def3_row29_col2" class="data row29 col2" >✅MET</td>      <td id="T_0def3_row29_col3" class="data row29 col3" >#BACHELORETTE</td>    </tr>    <tr>      <th id="T_0def3_level0_row30" class="row_heading level0 row30" >30</th>      <td id="T_0def3_row30_col0" class="data row30 col0" >TEALWALK</td>      <td id="T_0def3_row30_col1" class="data row30 col1" ></td>      <td id="T_0def3_row30_col2" class="data row30 col2" >crizMET</td>      <td id="T_0def3_row30_col3" class="data row30 col3" >RETURNED>>>>>>>>>>>>></td>    </tr>    <tr>      <th id="T_0def3_level0_row31" class="row_heading level0 row31" >31</th>      <td id="T_0def3_row31_col0" class="data row31 col0" >WALK”</td>      <td id="T_0def3_row31_col1" class="data row31 col1" ></td>      <td id="T_0def3_row31_col2" class="data row31 col2" >SOMETHING</td>      <td id="T_0def3_row31_col3" class="data row31 col3" >#ARETA</td>    </tr>    <tr>      <th id="T_0def3_level0_row32" class="row_heading level0 row32" >32</th>      <td id="T_0def3_row32_col0" class="data row32 col0" >#tALKpositive</td>      <td id="T_0def3_row32_col1" class="data row32 col1" ></td>      <td id="T_0def3_row32_col2" class="data row32 col2" >#METamplification</td>      <td id="T_0def3_row32_col3" class="data row32 col3" >DRETest</td>    </tr>    <tr>      <th id="T_0def3_level0_row33" class="row_heading level0 row33" >33</th>      <td id="T_0def3_row33_col0" class="data row33 col0" >#TALKPINK</td>      <td id="T_0def3_row33_col1" class="data row33 col1" ></td>      <td id="T_0def3_row33_col2" class="data row33 col2" >META</td>      <td id="T_0def3_row33_col3" class="data row33 col3" >SABOURETCardio</td>    </tr>    <tr>      <th id="T_0def3_level0_row34" class="row_heading level0 row34" >34</th>      <td id="T_0def3_row34_col0" class="data row34 col0" >TALKS</td>      <td id="T_0def3_row34_col1" class="data row34 col1" ></td>      <td id="T_0def3_row34_col2" class="data row34 col2" >t@METAvivor</td>      <td id="T_0def3_row34_col3" class="data row34 col3" >RETURNING</td>    </tr>    <tr>      <th id="T_0def3_level0_row35" class="row_heading level0 row35" >35</th>      <td id="T_0def3_row35_col0" class="data row35 col0" >SPACEWALK</td>      <td id="T_0def3_row35_col1" class="data row35 col1" ></td>      <td id="T_0def3_row35_col2" class="data row35 col2" >@METUPorg</td>      <td id="T_0def3_row35_col3" class="data row35 col3" >targets👉RET</td>    </tr>    <tr>      <th id="T_0def3_level0_row36" class="row_heading level0 row36" >36</th>      <td id="T_0def3_row36_col0" class="data row36 col0" ></td>      <td id="T_0def3_row36_col1" class="data row36 col1" ></td>      <td id="T_0def3_row36_col2" class="data row36 col2" >METAthriving</td>      <td id="T_0def3_row36_col3" class="data row36 col3" >and\xa0RET</td>    </tr>    <tr>      <th id="T_0def3_level0_row37" class="row_heading level0 row37" >37</th>      <td id="T_0def3_row37_col0" class="data row37 col0" ></td>      <td id="T_0def3_row37_col1" class="data row37 col1" ></td>      <td id="T_0def3_row37_col2" class="data row37 col2" >#METAvivor</td>      <td id="T_0def3_row37_col3" class="data row37 col3" >VITREORETINAL</td>    </tr>    <tr>      <th id="T_0def3_level0_row38" class="row_heading level0 row38" >38</th>      <td id="T_0def3_row38_col0" class="data row38 col0" ></td>      <td id="T_0def3_row38_col1" class="data row38 col1" ></td>      <td id="T_0def3_row38_col2" class="data row38 col2" >#PAM50MET</td>      <td id="T_0def3_row38_col3" class="data row38 col3" >#SCORETrial</td>    </tr>    <tr>      <th id="T_0def3_level0_row39" class="row_heading level0 row39" >39</th>      <td id="T_0def3_row39_col0" class="data row39 col0" ></td>      <td id="T_0def3_row39_col1" class="data row39 col1" ></td>      <td id="T_0def3_row39_col2" class="data row39 col2" >CABOMETYX</td>      <td id="T_0def3_row39_col3" class="data row39 col3" >RARETeen</td>    </tr>    <tr>      <th id="T_0def3_level0_row40" class="row_heading level0 row40" >40</th>      <td id="T_0def3_row40_col0" class="data row40 col0" ></td>      <td id="T_0def3_row40_col1" class="data row40 col1" ></td>      <td id="T_0def3_row40_col2" class="data row40 col2" >#METUPorg</td>      <td id="T_0def3_row40_col3" class="data row40 col3" >#LIBRETTO001</td>    </tr>    <tr>      <th id="T_0def3_level0_row41" class="row_heading level0 row41" >41</th>      <td id="T_0def3_row41_col0" class="data row41 col0" ></td>      <td id="T_0def3_row41_col1" class="data row41 col1" ></td>      <td id="T_0def3_row41_col2" class="data row41 col2" >#FIBROMET</td>      <td id="T_0def3_row41_col3" class="data row41 col3" >video*SECRET</td>    </tr>    <tr>      <th id="T_0def3_level0_row42" class="row_heading level0 row42" >42</th>      <td id="T_0def3_row42_col0" class="data row42 col0" ></td>      <td id="T_0def3_row42_col1" class="data row42 col1" ></td>      <td id="T_0def3_row42_col2" class="data row42 col2" >#PRIMETOWN2021</td>      <td id="T_0def3_row42_col3" class="data row42 col3" >EGFR|ALK|ROS1|RET|BRAF|MET</td>    </tr>    <tr>      <th id="T_0def3_level0_row43" class="row_heading level0 row43" >43</th>      <td id="T_0def3_row43_col0" class="data row43 col0" ></td>      <td id="T_0def3_row43_col1" class="data row43 col1" ></td>      <td id="T_0def3_row43_col2" class="data row43 col2" >#METsurv</td>      <td id="T_0def3_row43_col3" class="data row43 col3" >CUREToday</td>    </tr>    <tr>      <th id="T_0def3_level0_row44" class="row_heading level0 row44" >44</th>      <td id="T_0def3_row44_col0" class="data row44 col0" ></td>      <td id="T_0def3_row44_col1" class="data row44 col1" ></td>      <td id="T_0def3_row44_col2" class="data row44 col2" >8PMET</td>      <td id="T_0def3_row44_col3" class="data row44 col3" >#ENTRETENIMENTO</td>    </tr>    <tr>      <th id="T_0def3_level0_row45" class="row_heading level0 row45" >45</th>      <td id="T_0def3_row45_col0" class="data row45 col0" ></td>      <td id="T_0def3_row45_col1" class="data row45 col1" ></td>      <td id="T_0def3_row45_col2" class="data row45 col2" >friend☀️🔥@METUPUKorg</td>      <td id="T_0def3_row45_col3" class="data row45 col3" >#RETHINKPINK</td>    </tr>    <tr>      <th id="T_0def3_level0_row46" class="row_heading level0 row46" >46</th>      <td id="T_0def3_row46_col0" class="data row46 col0" ></td>      <td id="T_0def3_row46_col1" class="data row46 col1" ></td>      <td id="T_0def3_row46_col2" class="data row46 col2" >TCGA+METABRIC</td>      <td id="T_0def3_row46_col3" class="data row46 col3" >RETURNED</td>    </tr>    <tr>      <th id="T_0def3_level0_row47" class="row_heading level0 row47" >47</th>      <td id="T_0def3_row47_col0" class="data row47 col0" ></td>      <td id="T_0def3_row47_col1" class="data row47 col1" ></td>      <td id="T_0def3_row47_col2" class="data row47 col2" >#METUp</td>      <td id="T_0def3_row47_col3" class="data row47 col3" >ARETTA</td>    </tr>    <tr>      <th id="T_0def3_level0_row48" class="row_heading level0 row48" >48</th>      <td id="T_0def3_row48_col0" class="data row48 col0" ></td>      <td id="T_0def3_row48_col1" class="data row48 col1" ></td>      <td id="T_0def3_row48_col2" class="data row48 col2" >ADMET</td>      <td id="T_0def3_row48_col3" class="data row48 col3" >DIRETO</td>    </tr>    <tr>      <th id="T_0def3_level0_row49" class="row_heading level0 row49" >49</th>      <td id="T_0def3_row49_col0" class="data row49 col0" ></td>      <td id="T_0def3_row49_col1" class="data row49 col1" ></td>      <td id="T_0def3_row49_col2" class="data row49 col2" >#WELCOMETOOURCHRUCH</td>      <td id="T_0def3_row49_col3" class="data row49 col3" >RETREATMENT</td>    </tr>    <tr>      <th id="T_0def3_level0_row50" class="row_heading level0 row50" >50</th>      <td id="T_0def3_row50_col0" class="data row50 col0" ></td>      <td id="T_0def3_row50_col1" class="data row50 col1" ></td>      <td id="T_0def3_row50_col2" class="data row50 col2" >METavivor</td>      <td id="T_0def3_row50_col3" class="data row50 col3" >video~SECRET</td>    </tr>    <tr>      <th id="T_0def3_level0_row51" class="row_heading level0 row51" >51</th>      <td id="T_0def3_row51_col0" class="data row51 col0" ></td>      <td id="T_0def3_row51_col1" class="data row51 col1" ></td>      <td id="T_0def3_row51_col2" class="data row51 col2" >EGFR–MET</td>      <td id="T_0def3_row51_col3" class="data row51 col3" >@CURETODAY</td>    </tr>    <tr>      <th id="T_0def3_level0_row52" class="row_heading level0 row52" >52</th>      <td id="T_0def3_row52_col0" class="data row52 col0" ></td>      <td id="T_0def3_row52_col1" class="data row52 col1" ></td>      <td id="T_0def3_row52_col2" class="data row52 col2" >#METUPwithLungCancerDeath</td>      <td id="T_0def3_row52_col3" class="data row52 col3" >RETWEETING</td>    </tr>    <tr>      <th id="T_0def3_level0_row53" class="row_heading level0 row53" >53</th>      <td id="T_0def3_row53_col0" class="data row53 col0" ></td>      <td id="T_0def3_row53_col1" class="data row53 col1" ></td>      <td id="T_0def3_row53_col2" class="data row53 col2" >@METavivor</td>      <td id="T_0def3_row53_col3" class="data row53 col3" >#LIBRETTO</td>    </tr>    <tr>      <th id="T_0def3_level0_row54" class="row_heading level0 row54" >54</th>      <td id="T_0def3_row54_col0" class="data row54 col0" ></td>      <td id="T_0def3_row54_col1" class="data row54 col1" ></td>      <td id="T_0def3_row54_col2" class="data row54 col2" >#NWMETSCONF</td>      <td id="T_0def3_row54_col3" class="data row54 col3" >#RETalteredcancers</td>    </tr>    <tr>      <th id="T_0def3_level0_row55" class="row_heading level0 row55" >55</th>      <td id="T_0def3_row55_col0" class="data row55 col0" ></td>      <td id="T_0def3_row55_col1" class="data row55 col1" ></td>      <td id="T_0def3_row55_col2" class="data row55 col2" >METHOTREXATE</td>      <td id="T_0def3_row55_col3" class="data row55 col3" >#SECRET</td>    </tr>    <tr>      <th id="T_0def3_level0_row56" class="row_heading level0 row56" >56</th>      <td id="T_0def3_row56_col0" class="data row56 col0" ></td>      <td id="T_0def3_row56_col1" class="data row56 col1" ></td>      <td id="T_0def3_row56_col2" class="data row56 col2" >#SELIMETRY</td>      <td id="T_0def3_row56_col3" class="data row56 col3" >☆☆☆☆☆SECRET</td>    </tr>    <tr>      <th id="T_0def3_level0_row57" class="row_heading level0 row57" >57</th>      <td id="T_0def3_row57_col0" class="data row57 col0" ></td>      <td id="T_0def3_row57_col1" class="data row57 col1" ></td>      <td id="T_0def3_row57_col2" class="data row57 col2" >#METABRIC</td>      <td id="T_0def3_row57_col3" class="data row57 col3" >93RETY</td>    </tr>    <tr>      <th id="T_0def3_level0_row58" class="row_heading level0 row58" >58</th>      <td id="T_0def3_row58_col0" class="data row58 col0" ></td>      <td id="T_0def3_row58_col1" class="data row58 col1" ></td>      <td id="T_0def3_row58_col2" class="data row58 col2" >GAMETIME</td>      <td id="T_0def3_row58_col3" class="data row58 col3" >CALRETICULIN</td>    </tr>    <tr>      <th id="T_0def3_level0_row59" class="row_heading level0 row59" >59</th>      <td id="T_0def3_row59_col0" class="data row59 col0" ></td>      <td id="T_0def3_row59_col1" class="data row59 col1" ></td>      <td id="T_0def3_row59_col2" class="data row59 col2" >METS</td>      <td id="T_0def3_row59_col3" class="data row59 col3" >HRET</td>    </tr>    <tr>      <th id="T_0def3_level0_row60" class="row_heading level0 row60" >60</th>      <td id="T_0def3_row60_col0" class="data row60 col0" ></td>      <td id="T_0def3_row60_col1" class="data row60 col1" ></td>      <td id="T_0def3_row60_col2" class="data row60 col2" >unMET</td>      <td id="T_0def3_row60_col3" class="data row60 col3" >#AWOLRETREAT</td>    </tr>    <tr>      <th id="T_0def3_level0_row61" class="row_heading level0 row61" >61</th>      <td id="T_0def3_row61_col0" class="data row61 col0" ></td>      <td id="T_0def3_row61_col1" class="data row61 col1" ></td>      <td id="T_0def3_row61_col2" class="data row61 col2" >#METS</td>      <td id="T_0def3_row61_col3" class="data row61 col3" >INTERPRETATION</td>    </tr>    <tr>      <th id="T_0def3_level0_row62" class="row_heading level0 row62" >62</th>      <td id="T_0def3_row62_col0" class="data row62 col0" ></td>      <td id="T_0def3_row62_col1" class="data row62 col1" ></td>      <td id="T_0def3_row62_col2" class="data row62 col2" >COMET</td>      <td id="T_0def3_row62_col3" class="data row62 col3" >LORETTA</td>    </tr>    <tr>      <th id="T_0def3_level0_row63" class="row_heading level0 row63" >63</th>      <td id="T_0def3_row63_col0" class="data row63 col0" ></td>      <td id="T_0def3_row63_col1" class="data row63 col1" ></td>      <td id="T_0def3_row63_col2" class="data row63 col2" >PROMETRIKA</td>      <td id="T_0def3_row63_col3" class="data row63 col3" >#CABARET</td>    </tr>    <tr>      <th id="T_0def3_level0_row64" class="row_heading level0 row64" >64</th>      <td id="T_0def3_row64_col0" class="data row64 col0" ></td>      <td id="T_0def3_row64_col1" class="data row64 col1" ></td>      <td id="T_0def3_row64_col2" class="data row64 col2" >PROMETEEE</td>      <td id="T_0def3_row64_col3" class="data row64 col3" >CABARET</td>    </tr>    <tr>      <th id="T_0def3_level0_row65" class="row_heading level0 row65" >65</th>      <td id="T_0def3_row65_col0" class="data row65 col0" ></td>      <td id="T_0def3_row65_col1" class="data row65 col1" ></td>      <td id="T_0def3_row65_col2" class="data row65 col2" >#METAthriving</td>      <td id="T_0def3_row65_col3" class="data row65 col3" >@SABOURETCardio</td>    </tr>    <tr>      <th id="T_0def3_level0_row66" class="row_heading level0 row66" >66</th>      <td id="T_0def3_row66_col0" class="data row66 col0" ></td>      <td id="T_0def3_row66_col1" class="data row66 col1" ></td>      <td id="T_0def3_row66_col2" class="data row66 col2" >@CJMETA</td>      <td id="T_0def3_row66_col3" class="data row66 col3" >LIBRETTO</td>    </tr>    <tr>      <th id="T_0def3_level0_row67" class="row_heading level0 row67" >67</th>      <td id="T_0def3_row67_col0" class="data row67 col0" ></td>      <td id="T_0def3_row67_col1" class="data row67 col1" ></td>      <td id="T_0def3_row67_col2" class="data row67 col2" >#METExon14Skip</td>      <td id="T_0def3_row67_col3" class="data row67 col3" >#UnidosPorUnRETO</td>    </tr>    <tr>      <th id="T_0def3_level0_row68" class="row_heading level0 row68" >68</th>      <td id="T_0def3_row68_col0" class="data row68 col0" ></td>      <td id="T_0def3_row68_col1" class="data row68 col1" ></td>      <td id="T_0def3_row68_col2" class="data row68 col2" >METexon14</td>      <td id="T_0def3_row68_col3" class="data row68 col3" >COLORRETAL</td>    </tr>    <tr>      <th id="T_0def3_level0_row69" class="row_heading level0 row69" >69</th>      <td id="T_0def3_row69_col0" class="data row69 col0" ></td>      <td id="T_0def3_row69_col1" class="data row69 col1" ></td>      <td id="T_0def3_row69_col2" class="data row69 col2" >@METUporg</td>      <td id="T_0def3_row69_col3" class="data row69 col3" >RETRASOS</td>    </tr>    <tr>      <th id="T_0def3_level0_row70" class="row_heading level0 row70" >70</th>      <td id="T_0def3_row70_col0" class="data row70 col0" ></td>      <td id="T_0def3_row70_col1" class="data row70 col1" ></td>      <td id="T_0def3_row70_col2" class="data row70 col2" >\u2066@METUPorg\u2069</td>      <td id="T_0def3_row70_col3" class="data row70 col3" >2RETfs7</td>    </tr>    <tr>      <th id="T_0def3_level0_row71" class="row_heading level0 row71" >71</th>      <td id="T_0def3_row71_col0" class="data row71 col0" ></td>      <td id="T_0def3_row71_col1" class="data row71 col1" ></td>      <td id="T_0def3_row71_col2" class="data row71 col2" >METamp</td>      <td id="T_0def3_row71_col3" class="data row71 col3" >#MORETHANPINK</td>    </tr>    <tr>      <th id="T_0def3_level0_row72" class="row_heading level0 row72" >72</th>      <td id="T_0def3_row72_col0" class="data row72 col0" ></td>      <td id="T_0def3_row72_col1" class="data row72 col1" ></td>      <td id="T_0def3_row72_col2" class="data row72 col2" >#METskill</td>      <td id="T_0def3_row72_col3" class="data row72 col3" >#RARETalk</td>    </tr>    <tr>      <th id="T_0def3_level0_row73" class="row_heading level0 row73" >73</th>      <td id="T_0def3_row73_col0" class="data row73 col0" ></td>      <td id="T_0def3_row73_col1" class="data row73 col1" ></td>      <td id="T_0def3_row73_col2" class="data row73 col2" >SOMETHING*</td>      <td id="T_0def3_row73_col3" class="data row73 col3" >B2PiY29d5yRETiYZ9bIyHYQYVaRaE</td>    </tr>    <tr>      <th id="T_0def3_level0_row74" class="row_heading level0 row74" >74</th>      <td id="T_0def3_row74_col0" class="data row74 col0" ></td>      <td id="T_0def3_row74_col1" class="data row74 col1" ></td>      <td id="T_0def3_row74_col2" class="data row74 col2" >COMETs</td>      <td id="T_0def3_row74_col3" class="data row74 col3" >#CURETalksCancer</td>    </tr>    <tr>      <th id="T_0def3_level0_row75" class="row_heading level0 row75" >75</th>      <td id="T_0def3_row75_col0" class="data row75 col0" ></td>      <td id="T_0def3_row75_col1" class="data row75 col1" ></td>      <td id="T_0def3_row75_col2" class="data row75 col2" >MABOMET</td>      <td id="T_0def3_row75_col3" class="data row75 col3" >CORRETA</td>    </tr>    <tr>      <th id="T_0def3_level0_row76" class="row_heading level0 row76" >76</th>      <td id="T_0def3_row76_col0" class="data row76 col0" ></td>      <td id="T_0def3_row76_col1" class="data row76 col1" ></td>      <td id="T_0def3_row76_col2" class="data row76 col2" >METAVIVOR</td>      <td id="T_0def3_row76_col3" class="data row76 col3" >RETREAT</td>    </tr>    <tr>      <th id="T_0def3_level0_row77" class="row_heading level0 row77" >77</th>      <td id="T_0def3_row77_col0" class="data row77 col0" ></td>      <td id="T_0def3_row77_col1" class="data row77 col1" ></td>      <td id="T_0def3_row77_col2" class="data row77 col2" >METex14del</td>      <td id="T_0def3_row77_col3" class="data row77 col3" >@ABHURET</td>    </tr>    <tr>      <th id="T_0def3_level0_row78" class="row_heading level0 row78" >78</th>      <td id="T_0def3_row78_col0" class="data row78 col0" ></td>      <td id="T_0def3_row78_col1" class="data row78 col1" ></td>      <td id="T_0def3_row78_col2" class="data row78 col2" >#METUP2016Retreat</td>      <td id="T_0def3_row78_col3" class="data row78 col3" >RETinhibitor</td>    </tr>    <tr>      <th id="T_0def3_level0_row79" class="row_heading level0 row79" >79</th>      <td id="T_0def3_row79_col0" class="data row79 col0" ></td>      <td id="T_0def3_row79_col1" class="data row79 col1" ></td>      <td id="T_0def3_row79_col2" class="data row79 col2" >@FEMETREAC1</td>      <td id="T_0def3_row79_col3" class="data row79 col3" >RETrenegades</td>    </tr>    <tr>      <th id="T_0def3_level0_row80" class="row_heading level0 row80" >80</th>      <td id="T_0def3_row80_col0" class="data row80 col0" ></td>      <td id="T_0def3_row80_col1" class="data row80 col1" ></td>      <td id="T_0def3_row80_col2" class="data row80 col2" >ANGIOMET</td>      <td id="T_0def3_row80_col3" class="data row80 col3" >PRETTY</td>    </tr>    <tr>      <th id="T_0def3_level0_row81" class="row_heading level0 row81" >81</th>      <td id="T_0def3_row81_col0" class="data row81 col0" ></td>      <td id="T_0def3_row81_col1" class="data row81 col1" ></td>      <td id="T_0def3_row81_col2" class="data row81 col2" >⛔️MET</td>      <td id="T_0def3_row81_col3" class="data row81 col3" >RETi</td>    </tr>    <tr>      <th id="T_0def3_level0_row82" class="row_heading level0 row82" >82</th>      <td id="T_0def3_row82_col0" class="data row82 col0" ></td>      <td id="T_0def3_row82_col1" class="data row82 col1" ></td>      <td id="T_0def3_row82_col2" class="data row82 col2" >@COMET2017Indy</td>      <td id="T_0def3_row82_col3" class="data row82 col3" >🔶RET</td>    </tr>    <tr>      <th id="T_0def3_level0_row83" class="row_heading level0 row83" >83</th>      <td id="T_0def3_row83_col0" class="data row83 col0" ></td>      <td id="T_0def3_row83_col1" class="data row83 col1" ></td>      <td id="T_0def3_row83_col2" class="data row83 col2" >METAVivor</td>      <td id="T_0def3_row83_col3" class="data row83 col3" >RETWEET</td>    </tr>    <tr>      <th id="T_0def3_level0_row84" class="row_heading level0 row84" >84</th>      <td id="T_0def3_row84_col0" class="data row84 col0" ></td>      <td id="T_0def3_row84_col1" class="data row84 col1" ></td>      <td id="T_0def3_row84_col2" class="data row84 col2" >PROMETHEUS</td>      <td id="T_0def3_row84_col3" class="data row84 col3" >margaRET</td>    </tr>    <tr>      <th id="T_0def3_level0_row85" class="row_heading level0 row85" >85</th>      <td id="T_0def3_row85_col0" class="data row85 col0" ></td>      <td id="T_0def3_row85_col1" class="data row85 col1" ></td>      <td id="T_0def3_row85_col2" class="data row85 col2" >2@8PMET</td>      <td id="T_0def3_row85_col3" class="data row85 col3" >RETRASAR</td>    </tr>    <tr>      <th id="T_0def3_level0_row86" class="row_heading level0 row86" >86</th>      <td id="T_0def3_row86_col0" class="data row86 col0" ></td>      <td id="T_0def3_row86_col1" class="data row86 col1" ></td>      <td id="T_0def3_row86_col2" class="data row86 col2" >METUPUK</td>      <td id="T_0def3_row86_col3" class="data row86 col3" >RETHINKING</td>    </tr>    <tr>      <th id="T_0def3_level0_row87" class="row_heading level0 row87" >87</th>      <td id="T_0def3_row87_col0" class="data row87 col0" ></td>      <td id="T_0def3_row87_col1" class="data row87 col1" ></td>      <td id="T_0def3_row87_col2" class="data row87 col2" >#METASTATICBREASTCANCER</td>      <td id="T_0def3_row87_col3" class="data row87 col3" >#WEARETHEHERD</td>    </tr>    <tr>      <th id="T_0def3_level0_row88" class="row_heading level0 row88" >88</th>      <td id="T_0def3_row88_col0" class="data row88 col0" ></td>      <td id="T_0def3_row88_col1" class="data row88 col1" ></td>      <td id="T_0def3_row88_col2" class="data row88 col2" >\u2066\u2066@METUPorg\u2069</td>      <td id="T_0def3_row88_col3" class="data row88 col3" >#NGSforRET</td>    </tr>    <tr>      <th id="T_0def3_level0_row89" class="row_heading level0 row89" >89</th>      <td id="T_0def3_row89_col0" class="data row89 col0" ></td>      <td id="T_0def3_row89_col1" class="data row89 col1" ></td>      <td id="T_0def3_row89_col2" class="data row89 col2" >1MET</td>      <td id="T_0def3_row89_col3" class="data row89 col3" >🧬RET</td>    </tr>    <tr>      <th id="T_0def3_level0_row90" class="row_heading level0 row90" >90</th>      <td id="T_0def3_row90_col0" class="data row90 col0" ></td>      <td id="T_0def3_row90_col1" class="data row90 col1" ></td>      <td id="T_0def3_row90_col2" class="data row90 col2" >METEOR</td>      <td id="T_0def3_row90_col3" class="data row90 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row91" class="row_heading level0 row91" >91</th>      <td id="T_0def3_row91_col0" class="data row91 col0" ></td>      <td id="T_0def3_row91_col1" class="data row91 col1" ></td>      <td id="T_0def3_row91_col2" class="data row91 col2" >#HaveYouMETCancer</td>      <td id="T_0def3_row91_col3" class="data row91 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row92" class="row_heading level0 row92" >92</th>      <td id="T_0def3_row92_col0" class="data row92 col0" ></td>      <td id="T_0def3_row92_col1" class="data row92 col1" ></td>      <td id="T_0def3_row92_col2" class="data row92 col2" >GEOMETRY</td>      <td id="T_0def3_row92_col3" class="data row92 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row93" class="row_heading level0 row93" >93</th>      <td id="T_0def3_row93_col0" class="data row93 col0" ></td>      <td id="T_0def3_row93_col1" class="data row93 col1" ></td>      <td id="T_0def3_row93_col2" class="data row93 col2" >👉METUP👈</td>      <td id="T_0def3_row93_col3" class="data row93 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row94" class="row_heading level0 row94" >94</th>      <td id="T_0def3_row94_col0" class="data row94 col0" ></td>      <td id="T_0def3_row94_col1" class="data row94 col1" ></td>      <td id="T_0def3_row94_col2" class="data row94 col2" >@METAvivorResear</td>      <td id="T_0def3_row94_col3" class="data row94 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row95" class="row_heading level0 row95" >95</th>      <td id="T_0def3_row95_col0" class="data row95 col0" ></td>      <td id="T_0def3_row95_col1" class="data row95 col1" ></td>      <td id="T_0def3_row95_col2" class="data row95 col2" >#METSmonday</td>      <td id="T_0def3_row95_col3" class="data row95 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row96" class="row_heading level0 row96" >96</th>      <td id="T_0def3_row96_col0" class="data row96 col0" ></td>      <td id="T_0def3_row96_col1" class="data row96 col1" ></td>      <td id="T_0def3_row96_col2" class="data row96 col2" >#LBBCMETSConf</td>      <td id="T_0def3_row96_col3" class="data row96 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row97" class="row_heading level0 row97" >97</th>      <td id="T_0def3_row97_col0" class="data row97 col0" ></td>      <td id="T_0def3_row97_col1" class="data row97 col1" ></td>      <td id="T_0def3_row97_col2" class="data row97 col2" >OLIGOMET</td>      <td id="T_0def3_row97_col3" class="data row97 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row98" class="row_heading level0 row98" >98</th>      <td id="T_0def3_row98_col0" class="data row98 col0" ></td>      <td id="T_0def3_row98_col1" class="data row98 col1" ></td>      <td id="T_0def3_row98_col2" class="data row98 col2" >#METSURV</td>      <td id="T_0def3_row98_col3" class="data row98 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row99" class="row_heading level0 row99" >99</th>      <td id="T_0def3_row99_col0" class="data row99 col0" ></td>      <td id="T_0def3_row99_col1" class="data row99 col1" ></td>      <td id="T_0def3_row99_col2" class="data row99 col2" >#TMETuesday</td>      <td id="T_0def3_row99_col3" class="data row99 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row100" class="row_heading level0 row100" >100</th>      <td id="T_0def3_row100_col0" class="data row100 col0" ></td>      <td id="T_0def3_row100_col1" class="data row100 col1" ></td>      <td id="T_0def3_row100_col2" class="data row100 col2" >METUPorg</td>      <td id="T_0def3_row100_col3" class="data row100 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row101" class="row_heading level0 row101" >101</th>      <td id="T_0def3_row101_col0" class="data row101 col0" ></td>      <td id="T_0def3_row101_col1" class="data row101 col1" ></td>      <td id="T_0def3_row101_col2" class="data row101 col2" >FEMETREAC1</td>      <td id="T_0def3_row101_col3" class="data row101 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row102" class="row_heading level0 row102" >102</th>      <td id="T_0def3_row102_col0" class="data row102 col0" ></td>      <td id="T_0def3_row102_col1" class="data row102 col1" ></td>      <td id="T_0def3_row102_col2" class="data row102 col2" >#COMETtrial</td>      <td id="T_0def3_row102_col3" class="data row102 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row103" class="row_heading level0 row103" >103</th>      <td id="T_0def3_row103_col0" class="data row103 col0" ></td>      <td id="T_0def3_row103_col1" class="data row103 col1" ></td>      <td id="T_0def3_row103_col2" class="data row103 col2" >#METamp</td>      <td id="T_0def3_row103_col3" class="data row103 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row104" class="row_heading level0 row104" >104</th>      <td id="T_0def3_row104_col0" class="data row104 col0" ></td>      <td id="T_0def3_row104_col1" class="data row104 col1" ></td>      <td id="T_0def3_row104_col2" class="data row104 col2" >PROMET</td>      <td id="T_0def3_row104_col3" class="data row104 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row105" class="row_heading level0 row105" >105</th>      <td id="T_0def3_row105_col0" class="data row105 col0" ></td>      <td id="T_0def3_row105_col1" class="data row105 col1" ></td>      <td id="T_0def3_row105_col2" class="data row105 col2" >@COMETinitiative</td>      <td id="T_0def3_row105_col3" class="data row105 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row106" class="row_heading level0 row106" >106</th>      <td id="T_0def3_row106_col0" class="data row106 col0" ></td>      <td id="T_0def3_row106_col1" class="data row106 col1" ></td>      <td id="T_0def3_row106_col2" class="data row106 col2" >COMET3</td>      <td id="T_0def3_row106_col3" class="data row106 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row107" class="row_heading level0 row107" >107</th>      <td id="T_0def3_row107_col0" class="data row107 col0" ></td>      <td id="T_0def3_row107_col1" class="data row107 col1" ></td>      <td id="T_0def3_row107_col2" class="data row107 col2" ><i>MET<</td>      <td id="T_0def3_row107_col3" class="data row107 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row108" class="row_heading level0 row108" >108</th>      <td id="T_0def3_row108_col0" class="data row108 col0" ></td>      <td id="T_0def3_row108_col1" class="data row108 col1" ></td>      <td id="T_0def3_row108_col2" class="data row108 col2" >#METUPORG</td>      <td id="T_0def3_row108_col3" class="data row108 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row109" class="row_heading level0 row109" >109</th>      <td id="T_0def3_row109_col0" class="data row109 col0" ></td>      <td id="T_0def3_row109_col1" class="data row109 col1" ></td>      <td id="T_0def3_row109_col2" class="data row109 col2" >#ex14MET</td>      <td id="T_0def3_row109_col3" class="data row109 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row110" class="row_heading level0 row110" >110</th>      <td id="T_0def3_row110_col0" class="data row110 col0" ></td>      <td id="T_0def3_row110_col1" class="data row110 col1" ></td>      <td id="T_0def3_row110_col2" class="data row110 col2" >METExon14</td>      <td id="T_0def3_row110_col3" class="data row110 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row111" class="row_heading level0 row111" >111</th>      <td id="T_0def3_row111_col0" class="data row111 col0" ></td>      <td id="T_0def3_row111_col1" class="data row111 col1" ></td>      <td id="T_0def3_row111_col2" class="data row111 col2" >METASTASIS</td>      <td id="T_0def3_row111_col3" class="data row111 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row112" class="row_heading level0 row112" >112</th>      <td id="T_0def3_row112_col0" class="data row112 col0" ></td>      <td id="T_0def3_row112_col1" class="data row112 col1" ></td>      <td id="T_0def3_row112_col2" class="data row112 col2" >#METs</td>      <td id="T_0def3_row112_col3" class="data row112 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row113" class="row_heading level0 row113" >113</th>      <td id="T_0def3_row113_col0" class="data row113 col0" ></td>      <td id="T_0def3_row113_col1" class="data row113 col1" ></td>      <td id="T_0def3_row113_col2" class="data row113 col2" >METex14</td>      <td id="T_0def3_row113_col3" class="data row113 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row114" class="row_heading level0 row114" >114</th>      <td id="T_0def3_row114_col0" class="data row114 col0" ></td>      <td id="T_0def3_row114_col1" class="data row114 col1" ></td>      <td id="T_0def3_row114_col2" class="data row114 col2" >#METUP’s</td>      <td id="T_0def3_row114_col3" class="data row114 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row115" class="row_heading level0 row115" >115</th>      <td id="T_0def3_row115_col0" class="data row115 col0" ></td>      <td id="T_0def3_row115_col1" class="data row115 col1" ></td>      <td id="T_0def3_row115_col2" class="data row115 col2" >EGFR|ALK|ROS1|RET|BRAF|MET</td>      <td id="T_0def3_row115_col3" class="data row115 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row116" class="row_heading level0 row116" >116</th>      <td id="T_0def3_row116_col0" class="data row116 col0" ></td>      <td id="T_0def3_row116_col1" class="data row116 col1" ></td>      <td id="T_0def3_row116_col2" class="data row116 col2" >#TAKEMETOCHURCH</td>      <td id="T_0def3_row116_col3" class="data row116 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row117" class="row_heading level0 row117" >117</th>      <td id="T_0def3_row117_col0" class="data row117 col0" ></td>      <td id="T_0def3_row117_col1" class="data row117 col1" ></td>      <td id="T_0def3_row117_col2" class="data row117 col2" >#dcisCOMETstudy</td>      <td id="T_0def3_row117_col3" class="data row117 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row118" class="row_heading level0 row118" >118</th>      <td id="T_0def3_row118_col0" class="data row118 col0" ></td>      <td id="T_0def3_row118_col1" class="data row118 col1" ></td>      <td id="T_0def3_row118_col2" class="data row118 col2" >METs</td>      <td id="T_0def3_row118_col3" class="data row118 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row119" class="row_heading level0 row119" >119</th>      <td id="T_0def3_row119_col0" class="data row119 col0" ></td>      <td id="T_0def3_row119_col1" class="data row119 col1" ></td>      <td id="T_0def3_row119_col2" class="data row119 col2" >#BCMETS</td>      <td id="T_0def3_row119_col3" class="data row119 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row120" class="row_heading level0 row120" >120</th>      <td id="T_0def3_row120_col0" class="data row120 col0" ></td>      <td id="T_0def3_row120_col1" class="data row120 col1" ></td>      <td id="T_0def3_row120_col2" class="data row120 col2" >METHYLATION</td>      <td id="T_0def3_row120_col3" class="data row120 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row121" class="row_heading level0 row121" >121</th>      <td id="T_0def3_row121_col0" class="data row121 col0" ></td>      <td id="T_0def3_row121_col1" class="data row121 col1" ></td>      <td id="T_0def3_row121_col2" class="data row121 col2" >🔹MET</td>      <td id="T_0def3_row121_col3" class="data row121 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row122" class="row_heading level0 row122" >122</th>      <td id="T_0def3_row122_col0" class="data row122 col0" ></td>      <td id="T_0def3_row122_col1" class="data row122 col1" ></td>      <td id="T_0def3_row122_col2" class="data row122 col2" >9PMET</td>      <td id="T_0def3_row122_col3" class="data row122 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row123" class="row_heading level0 row123" >123</th>      <td id="T_0def3_row123_col0" class="data row123 col0" ></td>      <td id="T_0def3_row123_col1" class="data row123 col1" ></td>      <td id="T_0def3_row123_col2" class="data row123 col2" >#METexon14</td>      <td id="T_0def3_row123_col3" class="data row123 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row124" class="row_heading level0 row124" >124</th>      <td id="T_0def3_row124_col0" class="data row124 col0" ></td>      <td id="T_0def3_row124_col1" class="data row124 col1" ></td>      <td id="T_0def3_row124_col2" class="data row124 col2" >geoMETry</td>      <td id="T_0def3_row124_col3" class="data row124 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row125" class="row_heading level0 row125" >125</th>      <td id="T_0def3_row125_col0" class="data row125 col0" ></td>      <td id="T_0def3_row125_col1" class="data row125 col1" ></td>      <td id="T_0def3_row125_col2" class="data row125 col2" >iMET</td>      <td id="T_0def3_row125_col3" class="data row125 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row126" class="row_heading level0 row126" >126</th>      <td id="T_0def3_row126_col0" class="data row126 col0" ></td>      <td id="T_0def3_row126_col1" class="data row126 col1" ></td>      <td id="T_0def3_row126_col2" class="data row126 col2" >ZOMETA</td>      <td id="T_0def3_row126_col3" class="data row126 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row127" class="row_heading level0 row127" >127</th>      <td id="T_0def3_row127_col0" class="data row127 col0" ></td>      <td id="T_0def3_row127_col1" class="data row127 col1" ></td>      <td id="T_0def3_row127_col2" class="data row127 col2" >REMETY</td>      <td id="T_0def3_row127_col3" class="data row127 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row128" class="row_heading level0 row128" >128</th>      <td id="T_0def3_row128_col0" class="data row128 col0" ></td>      <td id="T_0def3_row128_col1" class="data row128 col1" ></td>      <td id="T_0def3_row128_col2" class="data row128 col2" >abroad👇🏼@METUPorg</td>      <td id="T_0def3_row128_col3" class="data row128 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row129" class="row_heading level0 row129" >129</th>      <td id="T_0def3_row129_col0" class="data row129 col0" ></td>      <td id="T_0def3_row129_col1" class="data row129 col1" ></td>      <td id="T_0def3_row129_col2" class="data row129 col2" >“METUP</td>      <td id="T_0def3_row129_col3" class="data row129 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row130" class="row_heading level0 row130" >130</th>      <td id="T_0def3_row130_col0" class="data row130 col0" ></td>      <td id="T_0def3_row130_col1" class="data row130 col1" ></td>      <td id="T_0def3_row130_col2" class="data row130 col2" >@METUP</td>      <td id="T_0def3_row130_col3" class="data row130 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row131" class="row_heading level0 row131" >131</th>      <td id="T_0def3_row131_col0" class="data row131 col0" ></td>      <td id="T_0def3_row131_col1" class="data row131 col1" ></td>      <td id="T_0def3_row131_col2" class="data row131 col2" >ENZAMET</td>      <td id="T_0def3_row131_col3" class="data row131 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row132" class="row_heading level0 row132" >132</th>      <td id="T_0def3_row132_col0" class="data row132 col0" ></td>      <td id="T_0def3_row132_col1" class="data row132 col1" ></td>      <td id="T_0def3_row132_col2" class="data row132 col2" >6PMET</td>      <td id="T_0def3_row132_col3" class="data row132 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row133" class="row_heading level0 row133" >133</th>      <td id="T_0def3_row133_col0" class="data row133 col0" ></td>      <td id="T_0def3_row133_col1" class="data row133 col1" ></td>      <td id="T_0def3_row133_col2" class="data row133 col2" >#COMET</td>      <td id="T_0def3_row133_col3" class="data row133 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row134" class="row_heading level0 row134" >134</th>      <td id="T_0def3_row134_col0" class="data row134 col0" ></td>      <td id="T_0def3_row134_col1" class="data row134 col1" ></td>      <td id="T_0def3_row134_col2" class="data row134 col2" >DEXAMETHASONE</td>      <td id="T_0def3_row134_col3" class="data row134 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row135" class="row_heading level0 row135" >135</th>      <td id="T_0def3_row135_col0" class="data row135 col0" ></td>      <td id="T_0def3_row135_col1" class="data row135 col1" ></td>      <td id="T_0def3_row135_col2" class="data row135 col2" >#METAvivor’s</td>      <td id="T_0def3_row135_col3" class="data row135 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row136" class="row_heading level0 row136" >136</th>      <td id="T_0def3_row136_col0" class="data row136 col0" ></td>      <td id="T_0def3_row136_col1" class="data row136 col1" ></td>      <td id="T_0def3_row136_col2" class="data row136 col2" >PRIMETEST</td>      <td id="T_0def3_row136_col3" class="data row136 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row137" class="row_heading level0 row137" >137</th>      <td id="T_0def3_row137_col0" class="data row137 col0" ></td>      <td id="T_0def3_row137_col1" class="data row137 col1" ></td>      <td id="T_0def3_row137_col2" class="data row137 col2" >#METster</td>      <td id="T_0def3_row137_col3" class="data row137 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row138" class="row_heading level0 row138" >138</th>      <td id="T_0def3_row138_col0" class="data row138 col0" ></td>      <td id="T_0def3_row138_col1" class="data row138 col1" ></td>      <td id="T_0def3_row138_col2" class="data row138 col2" >PROMETEO</td>      <td id="T_0def3_row138_col3" class="data row138 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row139" class="row_heading level0 row139" >139</th>      <td id="T_0def3_row139_col0" class="data row139 col0" ></td>      <td id="T_0def3_row139_col1" class="data row139 col1" ></td>      <td id="T_0def3_row139_col2" class="data row139 col2" >@Majormac1#METUP</td>      <td id="T_0def3_row139_col3" class="data row139 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row140" class="row_heading level0 row140" >140</th>      <td id="T_0def3_row140_col0" class="data row140 col0" ></td>      <td id="T_0def3_row140_col1" class="data row140 col1" ></td>      <td id="T_0def3_row140_col2" class="data row140 col2" >MIMETIC</td>      <td id="T_0def3_row140_col3" class="data row140 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row141" class="row_heading level0 row141" >141</th>      <td id="T_0def3_row141_col0" class="data row141 col0" ></td>      <td id="T_0def3_row141_col1" class="data row141 col1" ></td>      <td id="T_0def3_row141_col2" class="data row141 col2" >#METSTIVUS18</td>      <td id="T_0def3_row141_col3" class="data row141 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row142" class="row_heading level0 row142" >142</th>      <td id="T_0def3_row142_col0" class="data row142 col0" ></td>      <td id="T_0def3_row142_col1" class="data row142 col1" ></td>      <td id="T_0def3_row142_col2" class="data row142 col2" >METamps</td>      <td id="T_0def3_row142_col3" class="data row142 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row143" class="row_heading level0 row143" >143</th>      <td id="T_0def3_row143_col0" class="data row143 col0" ></td>      <td id="T_0def3_row143_col1" class="data row143 col1" ></td>      <td id="T_0def3_row143_col2" class="data row143 col2" >METUPUKorg</td>      <td id="T_0def3_row143_col3" class="data row143 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row144" class="row_heading level0 row144" >144</th>      <td id="T_0def3_row144_col0" class="data row144 col0" ></td>      <td id="T_0def3_row144_col1" class="data row144 col1" ></td>      <td id="T_0def3_row144_col2" class="data row144 col2" >2@METAvivor</td>      <td id="T_0def3_row144_col3" class="data row144 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row145" class="row_heading level0 row145" >145</th>      <td id="T_0def3_row145_col0" class="data row145 col0" ></td>      <td id="T_0def3_row145_col1" class="data row145 col1" ></td>      <td id="T_0def3_row145_col2" class="data row145 col2" >#METUPAsks</td>      <td id="T_0def3_row145_col3" class="data row145 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row146" class="row_heading level0 row146" >146</th>      <td id="T_0def3_row146_col0" class="data row146 col0" ></td>      <td id="T_0def3_row146_col1" class="data row146 col1" ></td>      <td id="T_0def3_row146_col2" class="data row146 col2" >1PMET</td>      <td id="T_0def3_row146_col3" class="data row146 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row147" class="row_heading level0 row147" >147</th>      <td id="T_0def3_row147_col0" class="data row147 col0" ></td>      <td id="T_0def3_row147_col1" class="data row147 col1" ></td>      <td id="T_0def3_row147_col2" class="data row147 col2" >@CKMETA</td>      <td id="T_0def3_row147_col3" class="data row147 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row148" class="row_heading level0 row148" >148</th>      <td id="T_0def3_row148_col0" class="data row148 col0" ></td>      <td id="T_0def3_row148_col1" class="data row148 col1" ></td>      <td id="T_0def3_row148_col2" class="data row148 col2" >HYPOMETHYLATING</td>      <td id="T_0def3_row148_col3" class="data row148 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row149" class="row_heading level0 row149" >149</th>      <td id="T_0def3_row149_col0" class="data row149 col0" ></td>      <td id="T_0def3_row149_col1" class="data row149 col1" ></td>      <td id="T_0def3_row149_col2" class="data row149 col2" >cMET</td>      <td id="T_0def3_row149_col3" class="data row149 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row150" class="row_heading level0 row150" >150</th>      <td id="T_0def3_row150_col0" class="data row150 col0" ></td>      <td id="T_0def3_row150_col1" class="data row150 col1" ></td>      <td id="T_0def3_row150_col2" class="data row150 col2" >METUP’s</td>      <td id="T_0def3_row150_col3" class="data row150 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row151" class="row_heading level0 row151" >151</th>      <td id="T_0def3_row151_col0" class="data row151 col0" ></td>      <td id="T_0def3_row151_col1" class="data row151 col1" ></td>      <td id="T_0def3_row151_col2" class="data row151 col2" >SOMETIMES</td>      <td id="T_0def3_row151_col3" class="data row151 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row152" class="row_heading level0 row152" >152</th>      <td id="T_0def3_row152_col0" class="data row152 col0" ></td>      <td id="T_0def3_row152_col1" class="data row152 col1" ></td>      <td id="T_0def3_row152_col2" class="data row152 col2" >vote🔥☎️🔥@METUPorg</td>      <td id="T_0def3_row152_col3" class="data row152 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row153" class="row_heading level0 row153" >153</th>      <td id="T_0def3_row153_col0" class="data row153 col0" ></td>      <td id="T_0def3_row153_col1" class="data row153 col1" ></td>      <td id="T_0def3_row153_col2" class="data row153 col2" >PARAMETERS</td>      <td id="T_0def3_row153_col3" class="data row153 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row154" class="row_heading level0 row154" >154</th>      <td id="T_0def3_row154_col0" class="data row154 col0" ></td>      <td id="T_0def3_row154_col1" class="data row154 col1" ></td>      <td id="T_0def3_row154_col2" class="data row154 col2" >BECLOMETHASONE</td>      <td id="T_0def3_row154_col3" class="data row154 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row155" class="row_heading level0 row155" >155</th>      <td id="T_0def3_row155_col0" class="data row155 col0" ></td>      <td id="T_0def3_row155_col1" class="data row155 col1" ></td>      <td id="T_0def3_row155_col2" class="data row155 col2" >METastasis</td>      <td id="T_0def3_row155_col3" class="data row155 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row156" class="row_heading level0 row156" >156</th>      <td id="T_0def3_row156_col0" class="data row156 col0" ></td>      <td id="T_0def3_row156_col1" class="data row156 col1" ></td>      <td id="T_0def3_row156_col2" class="data row156 col2" >#ENZAMET</td>      <td id="T_0def3_row156_col3" class="data row156 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row157" class="row_heading level0 row157" >157</th>      <td id="T_0def3_row157_col0" class="data row157 col0" ></td>      <td id="T_0def3_row157_col1" class="data row157 col1" ></td>      <td id="T_0def3_row157_col2" class="data row157 col2" >#METsters</td>      <td id="T_0def3_row157_col3" class="data row157 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row158" class="row_heading level0 row158" >158</th>      <td id="T_0def3_row158_col0" class="data row158 col0" ></td>      <td id="T_0def3_row158_col1" class="data row158 col1" ></td>      <td id="T_0def3_row158_col2" class="data row158 col2" >@METAvivior</td>      <td id="T_0def3_row158_col3" class="data row158 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row159" class="row_heading level0 row159" >159</th>      <td id="T_0def3_row159_col0" class="data row159 col0" ></td>      <td id="T_0def3_row159_col1" class="data row159 col1" ></td>      <td id="T_0def3_row159_col2" class="data row159 col2" >@METAvivor</td>      <td id="T_0def3_row159_col3" class="data row159 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row160" class="row_heading level0 row160" >160</th>      <td id="T_0def3_row160_col0" class="data row160 col0" ></td>      <td id="T_0def3_row160_col1" class="data row160 col1" ></td>      <td id="T_0def3_row160_col2" class="data row160 col2" >#METUPasks</td>      <td id="T_0def3_row160_col3" class="data row160 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row161" class="row_heading level0 row161" >161</th>      <td id="T_0def3_row161_col0" class="data row161 col0" ></td>      <td id="T_0def3_row161_col1" class="data row161 col1" ></td>      <td id="T_0def3_row161_col2" class="data row161 col2" >00PMET</td>      <td id="T_0def3_row161_col3" class="data row161 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row162" class="row_heading level0 row162" >162</th>      <td id="T_0def3_row162_col0" class="data row162 col0" ></td>      <td id="T_0def3_row162_col1" class="data row162 col1" ></td>      <td id="T_0def3_row162_col2" class="data row162 col2" >#COMETstudy</td>      <td id="T_0def3_row162_col3" class="data row162 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row163" class="row_heading level0 row163" >163</th>      <td id="T_0def3_row163_col0" class="data row163 col0" ></td>      <td id="T_0def3_row163_col1" class="data row163 col1" ></td>      <td id="T_0def3_row163_col2" class="data row163 col2" >METABRIC</td>      <td id="T_0def3_row163_col3" class="data row163 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row164" class="row_heading level0 row164" >164</th>      <td id="T_0def3_row164_col0" class="data row164 col0" ></td>      <td id="T_0def3_row164_col1" class="data row164 col1" ></td>      <td id="T_0def3_row164_col2" class="data row164 col2" >METAvivor</td>      <td id="T_0def3_row164_col3" class="data row164 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row165" class="row_heading level0 row165" >165</th>      <td id="T_0def3_row165_col0" class="data row165 col0" ></td>      <td id="T_0def3_row165_col1" class="data row165 col1" ></td>      <td id="T_0def3_row165_col2" class="data row165 col2" >METAvivors</td>      <td id="T_0def3_row165_col3" class="data row165 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row166" class="row_heading level0 row166" >166</th>      <td id="T_0def3_row166_col0" class="data row166 col0" ></td>      <td id="T_0def3_row166_col1" class="data row166 col1" ></td>      <td id="T_0def3_row166_col2" class="data row166 col2" >METASTASES</td>      <td id="T_0def3_row166_col3" class="data row166 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row167" class="row_heading level0 row167" >167</th>      <td id="T_0def3_row167_col0" class="data row167 col0" ></td>      <td id="T_0def3_row167_col1" class="data row167 col1" ></td>      <td id="T_0def3_row167_col2" class="data row167 col2" >#METAware</td>      <td id="T_0def3_row167_col3" class="data row167 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row168" class="row_heading level0 row168" >168</th>      <td id="T_0def3_row168_col0" class="data row168 col0" ></td>      <td id="T_0def3_row168_col1" class="data row168 col1" ></td>      <td id="T_0def3_row168_col2" class="data row168 col2" >KOMET</td>      <td id="T_0def3_row168_col3" class="data row168 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row169" class="row_heading level0 row169" >169</th>      <td id="T_0def3_row169_col0" class="data row169 col0" ></td>      <td id="T_0def3_row169_col1" class="data row169 col1" ></td>      <td id="T_0def3_row169_col2" class="data row169 col2" >#PROMETHEUS</td>      <td id="T_0def3_row169_col3" class="data row169 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row170" class="row_heading level0 row170" >170</th>      <td id="T_0def3_row170_col0" class="data row170 col0" ></td>      <td id="T_0def3_row170_col1" class="data row170 col1" ></td>      <td id="T_0def3_row170_col2" class="data row170 col2" >SOMETH</td>      <td id="T_0def3_row170_col3" class="data row170 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row171" class="row_heading level0 row171" >171</th>      <td id="T_0def3_row171_col0" class="data row171 col0" ></td>      <td id="T_0def3_row171_col1" class="data row171 col1" ></td>      <td id="T_0def3_row171_col2" class="data row171 col2" >#METAvivorMobile</td>      <td id="T_0def3_row171_col3" class="data row171 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row172" class="row_heading level0 row172" >172</th>      <td id="T_0def3_row172_col0" class="data row172 col0" ></td>      <td id="T_0def3_row172_col1" class="data row172 col1" ></td>      <td id="T_0def3_row172_col2" class="data row172 col2" >#@METUPorg</td>      <td id="T_0def3_row172_col3" class="data row172 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row173" class="row_heading level0 row173" >173</th>      <td id="T_0def3_row173_col0" class="data row173 col0" ></td>      <td id="T_0def3_row173_col1" class="data row173 col1" ></td>      <td id="T_0def3_row173_col2" class="data row173 col2" >@METUPUKorg</td>      <td id="T_0def3_row173_col3" class="data row173 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row174" class="row_heading level0 row174" >174</th>      <td id="T_0def3_row174_col0" class="data row174 col0" ></td>      <td id="T_0def3_row174_col1" class="data row174 col1" ></td>      <td id="T_0def3_row174_col2" class="data row174 col2" >in\xa0MET</td>      <td id="T_0def3_row174_col3" class="data row174 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row175" class="row_heading level0 row175" >175</th>      <td id="T_0def3_row175_col0" class="data row175 col0" ></td>      <td id="T_0def3_row175_col1" class="data row175 col1" ></td>      <td id="T_0def3_row175_col2" class="data row175 col2" >NonMETsers</td>      <td id="T_0def3_row175_col3" class="data row175 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row176" class="row_heading level0 row176" >176</th>      <td id="T_0def3_row176_col0" class="data row176 col0" ></td>      <td id="T_0def3_row176_col1" class="data row176 col1" ></td>      <td id="T_0def3_row176_col2" class="data row176 col2" >METex14s</td>      <td id="T_0def3_row176_col3" class="data row176 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row177" class="row_heading level0 row177" >177</th>      <td id="T_0def3_row177_col0" class="data row177 col0" ></td>      <td id="T_0def3_row177_col1" class="data row177 col1" ></td>      <td id="T_0def3_row177_col2" class="data row177 col2" >“@METUPorg</td>      <td id="T_0def3_row177_col3" class="data row177 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row178" class="row_heading level0 row178" >178</th>      <td id="T_0def3_row178_col0" class="data row178 col0" ></td>      <td id="T_0def3_row178_col1" class="data row178 col1" ></td>      <td id="T_0def3_row178_col2" class="data row178 col2" >2️⃣MET⛔️can</td>      <td id="T_0def3_row178_col3" class="data row178 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row179" class="row_heading level0 row179" >179</th>      <td id="T_0def3_row179_col0" class="data row179 col0" ></td>      <td id="T_0def3_row179_col1" class="data row179 col1" ></td>      <td id="T_0def3_row179_col2" class="data row179 col2" >antiMET</td>      <td id="T_0def3_row179_col3" class="data row179 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row180" class="row_heading level0 row180" >180</th>      <td id="T_0def3_row180_col0" class="data row180 col0" ></td>      <td id="T_0def3_row180_col1" class="data row180 col1" ></td>      <td id="T_0def3_row180_col2" class="data row180 col2" >particularly\xa0MET</td>      <td id="T_0def3_row180_col3" class="data row180 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row181" class="row_heading level0 row181" >181</th>      <td id="T_0def3_row181_col0" class="data row181 col0" ></td>      <td id="T_0def3_row181_col1" class="data row181 col1" ></td>      <td id="T_0def3_row181_col2" class="data row181 col2" >METUP</td>      <td id="T_0def3_row181_col3" class="data row181 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row182" class="row_heading level0 row182" >182</th>      <td id="T_0def3_row182_col0" class="data row182 col0" ></td>      <td id="T_0def3_row182_col1" class="data row182 col1" ></td>      <td id="T_0def3_row182_col2" class="data row182 col2" >METASTATIC</td>      <td id="T_0def3_row182_col3" class="data row182 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row183" class="row_heading level0 row183" >183</th>      <td id="T_0def3_row183_col0" class="data row183 col0" ></td>      <td id="T_0def3_row183_col1" class="data row183 col1" ></td>      <td id="T_0def3_row183_col2" class="data row183 col2" >#XPOMET</td>      <td id="T_0def3_row183_col3" class="data row183 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row184" class="row_heading level0 row184" >184</th>      <td id="T_0def3_row184_col0" class="data row184 col0" ></td>      <td id="T_0def3_row184_col1" class="data row184 col1" ></td>      <td id="T_0def3_row184_col2" class="data row184 col2" >METex14mut</td>      <td id="T_0def3_row184_col3" class="data row184 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row185" class="row_heading level0 row185" >185</th>      <td id="T_0def3_row185_col0" class="data row185 col0" ></td>      <td id="T_0def3_row185_col1" class="data row185 col1" ></td>      <td id="T_0def3_row185_col2" class="data row185 col2" >#METSMonday</td>      <td id="T_0def3_row185_col3" class="data row185 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row186" class="row_heading level0 row186" >186</th>      <td id="T_0def3_row186_col0" class="data row186 col0" ></td>      <td id="T_0def3_row186_col1" class="data row186 col1" ></td>      <td id="T_0def3_row186_col2" class="data row186 col2" >#ENZAMET…</td>      <td id="T_0def3_row186_col3" class="data row186 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row187" class="row_heading level0 row187" >187</th>      <td id="T_0def3_row187_col0" class="data row187 col0" ></td>      <td id="T_0def3_row187_col1" class="data row187 col1" ></td>      <td id="T_0def3_row187_col2" class="data row187 col2" >#METSMONDAY</td>      <td id="T_0def3_row187_col3" class="data row187 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row188" class="row_heading level0 row188" >188</th>      <td id="T_0def3_row188_col0" class="data row188 col0" ></td>      <td id="T_0def3_row188_col1" class="data row188 col1" ></td>      <td id="T_0def3_row188_col2" class="data row188 col2" >#METUPwithBCDeath</td>      <td id="T_0def3_row188_col3" class="data row188 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row189" class="row_heading level0 row189" >189</th>      <td id="T_0def3_row189_col0" class="data row189 col0" ></td>      <td id="T_0def3_row189_col1" class="data row189 col1" ></td>      <td id="T_0def3_row189_col2" class="data row189 col2" >PAM50MET</td>      <td id="T_0def3_row189_col3" class="data row189 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row190" class="row_heading level0 row190" >190</th>      <td id="T_0def3_row190_col0" class="data row190 col0" ></td>      <td id="T_0def3_row190_col1" class="data row190 col1" ></td>      <td id="T_0def3_row190_col2" class="data row190 col2" >#METAvivormobile</td>      <td id="T_0def3_row190_col3" class="data row190 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row191" class="row_heading level0 row191" >191</th>      <td id="T_0def3_row191_col0" class="data row191 col0" ></td>      <td id="T_0def3_row191_col1" class="data row191 col1" ></td>      <td id="T_0def3_row191_col2" class="data row191 col2" >SCIMET</td>      <td id="T_0def3_row191_col3" class="data row191 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row192" class="row_heading level0 row192" >192</th>      <td id="T_0def3_row192_col0" class="data row192 col0" ></td>      <td id="T_0def3_row192_col1" class="data row192 col1" ></td>      <td id="T_0def3_row192_col2" class="data row192 col2" >BAROMETER</td>      <td id="T_0def3_row192_col3" class="data row192 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row193" class="row_heading level0 row193" >193</th>      <td id="T_0def3_row193_col0" class="data row193 col0" ></td>      <td id="T_0def3_row193_col1" class="data row193 col1" ></td>      <td id="T_0def3_row193_col2" class="data row193 col2" >METsters</td>      <td id="T_0def3_row193_col3" class="data row193 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row194" class="row_heading level0 row194" >194</th>      <td id="T_0def3_row194_col0" class="data row194 col0" ></td>      <td id="T_0def3_row194_col1" class="data row194 col1" ></td>      <td id="T_0def3_row194_col2" class="data row194 col2" >#HOLDMETIGHT</td>      <td id="T_0def3_row194_col3" class="data row194 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row195" class="row_heading level0 row195" >195</th>      <td id="T_0def3_row195_col0" class="data row195 col0" ></td>      <td id="T_0def3_row195_col1" class="data row195 col1" ></td>      <td id="T_0def3_row195_col2" class="data row195 col2" >METampl</td>      <td id="T_0def3_row195_col3" class="data row195 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row196" class="row_heading level0 row196" >196</th>      <td id="T_0def3_row196_col0" class="data row196 col0" ></td>      <td id="T_0def3_row196_col1" class="data row196 col1" ></td>      <td id="T_0def3_row196_col2" class="data row196 col2" >METex14+</td>      <td id="T_0def3_row196_col3" class="data row196 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row197" class="row_heading level0 row197" >197</th>      <td id="T_0def3_row197_col0" class="data row197 col0" ></td>      <td id="T_0def3_row197_col1" class="data row197 col1" ></td>      <td id="T_0def3_row197_col2" class="data row197 col2" >#METUP</td>      <td id="T_0def3_row197_col3" class="data row197 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row198" class="row_heading level0 row198" >198</th>      <td id="T_0def3_row198_col0" class="data row198 col0" ></td>      <td id="T_0def3_row198_col1" class="data row198 col1" ></td>      <td id="T_0def3_row198_col2" class="data row198 col2" >METAstatic</td>      <td id="T_0def3_row198_col3" class="data row198 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row199" class="row_heading level0 row199" >199</th>      <td id="T_0def3_row199_col0" class="data row199 col0" ></td>      <td id="T_0def3_row199_col1" class="data row199 col1" ></td>      <td id="T_0def3_row199_col2" class="data row199 col2" >#METAthiver</td>      <td id="T_0def3_row199_col3" class="data row199 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row200" class="row_heading level0 row200" >200</th>      <td id="T_0def3_row200_col0" class="data row200 col0" ></td>      <td id="T_0def3_row200_col1" class="data row200 col1" ></td>      <td id="T_0def3_row200_col2" class="data row200 col2" >Katie🎄@METUPorg</td>      <td id="T_0def3_row200_col3" class="data row200 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row201" class="row_heading level0 row201" >201</th>      <td id="T_0def3_row201_col0" class="data row201 col0" ></td>      <td id="T_0def3_row201_col1" class="data row201 col1" ></td>      <td id="T_0def3_row201_col2" class="data row201 col2" >METHOD</td>      <td id="T_0def3_row201_col3" class="data row201 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row202" class="row_heading level0 row202" >202</th>      <td id="T_0def3_row202_col0" class="data row202 col0" ></td>      <td id="T_0def3_row202_col1" class="data row202 col1" ></td>      <td id="T_0def3_row202_col2" class="data row202 col2" >PMET</td>      <td id="T_0def3_row202_col3" class="data row202 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row203" class="row_heading level0 row203" >203</th>      <td id="T_0def3_row203_col0" class="data row203 col0" ></td>      <td id="T_0def3_row203_col1" class="data row203 col1" ></td>      <td id="T_0def3_row203_col2" class="data row203 col2" >data<METABRIC</td>      <td id="T_0def3_row203_col3" class="data row203 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row204" class="row_heading level0 row204" >204</th>      <td id="T_0def3_row204_col0" class="data row204 col0" ></td>      <td id="T_0def3_row204_col1" class="data row204 col1" ></td>      <td id="T_0def3_row204_col2" class="data row204 col2" >#METExon14</td>      <td id="T_0def3_row204_col3" class="data row204 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row205" class="row_heading level0 row205" >205</th>      <td id="T_0def3_row205_col0" class="data row205 col0" ></td>      <td id="T_0def3_row205_col1" class="data row205 col1" ></td>      <td id="T_0def3_row205_col2" class="data row205 col2" >METASTATICME</td>      <td id="T_0def3_row205_col3" class="data row205 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row206" class="row_heading level0 row206" >206</th>      <td id="T_0def3_row206_col0" class="data row206 col0" ></td>      <td id="T_0def3_row206_col1" class="data row206 col1" ></td>      <td id="T_0def3_row206_col2" class="data row206 col2" >ArcherMET</td>      <td id="T_0def3_row206_col3" class="data row206 col3" ></td>    </tr>    <tr>      <th id="T_0def3_level0_row207" class="row_heading level0 row207" >207</th>      <td id="T_0def3_row207_col0" class="data row207 col0" ></td>      <td id="T_0def3_row207_col1" class="data row207 col1" ></td>      <td id="T_0def3_row207_col2" class="data row207 col2" >FIBROMET</td>      <td id="T_0def3_row207_col3" class="data row207 col3" ></td>    </tr>  </tbody></table>'

In [57]:
#dfbiom1.to_csv(f"{path_base}liste_des_formes_exclues.csv", sep = ',')
#df00['date'] = pd.to_datetime(pd.to_datetime(df00['local_time']).dt.date)

#df00 = df00.drop_duplicates()
#df00.to_csv(f"{path_base}/corpus_tweets.csv", sep = ",")